# GIWAXS plotting notebook - plotting single images from loaded zarr datasets

## Imports

In [ ]:
# Imports:
import pathlib
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import mplcursors 
import xarray as xr
from tqdm.auto import tqdm

# For lmfit later
from scipy import optimize, signal
from lmfit import models

# Choose a colormap:
cmap = plt.cm.turbo
cmap.set_bad('black')

## Define & check paths

In [ ]:
def select_attrs(data_arrays_iterable, selected_attrs_dict):
    """
    Selects data arrays whose attributes match the specified values.

    Parameters:
    data_arrays_iterable: Iterable of xarray.DataArray objects.
    selected_attrs_dict: Dictionary where keys are attribute names and 
                         values are the attributes' desired values.

    Returns:
    List of xarray.DataArray objects that match the specified attributes.
    """    
    sublist = list(data_arrays_iterable)
    
    for attr_name, attr_values in selected_attrs_dict.items():
        sublist = [da.copy() for da in sublist if da.attrs[attr_name] in attr_values]
                
    return sublist

In [ ]:
# I like pathlib for its readability & checkability, it's also necessary for the loadSeries function later on
# Replace the paths with the ones relevant to your data, you can use the ".exists()" method to make sure you defined a path correctly
suitePath = pathlib.Path('/Users/andrew/Library/CloudStorage/OneDrive-UCB-O365/research/data_analysis/giwaxs_suite')
outPath = suitePath.joinpath('processed_data/xenocs/PS-AX_films_01')
zarrsPath = outPath.joinpath('zarrs')

In [ ]:
# List zarrs
sorted([f.name for f in zarrsPath.iterdir()])  # a way to list just the filenames and not the whole path

In [ ]:
# make quick dictionary to from sample id to sample name for plotting titles:
sn = {
    'PSs_01_quicktest_0_00000':'Neat PS, 2 min exp.',
    'PSs_01_0_00000':'Neat Polystyrene (PS)',
    'PStoA1s_1to1_0_00000':'PS:A1 1:1',
    'PStoA1s_3to1_0_00000':'PS:A1 3:1',
    'PStoA1s_9to1_0_00000':'PS:A1 9:1',
    'PStoA2s_1to1_0_00000':'PS:A2 1:1',
    'PStoA2s_3to1_0_00000':'PS:A2 3:1',
    'PStoA2s_9to1_0_00000':'PS:A2 9:1',
    'PStoA3s_1to1_0_00000':'PS:A3 1:1',
    'PStoA3s_3to1_0_00000':'PS:A3 3:1',
    'PStoA3s_9to1_0_00000':'PS:A3 9:1',
}

## Cartesian plots

### Raw data:

In [ ]:
filename = 'raw.zarr'
raw_DS = xr.open_zarr(zarrsPath.joinpath(filename)).compute()
raw_DS

In [ ]:
for DA in raw_DS.data_vars.values():
    cmin = DA.quantile(0.01)
    cmax = DA.quantile(0.99)
    ax = DA.sel(pix_y=slice(None,900)).plot.imshow(
        cmap=cmap, norm=plt.Normalize(cmin,cmax), origin='upper', figsize=(5,4), interpolation='antialiased')
    ax.axes.set(title=f'{sn[DA.name]}:\n Stitched & averaged raw image')
    ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=12)

    savePath = outPath.joinpath('raw_pix_plots_v1')
    savePath.mkdir(exist_ok=True)
    ax.figure.savefig(savePath.joinpath(f'{DA.name}_th{DA.incident_angle}.png'), dpi=150)
    
    # plt.show()
    plt.close('all')

### Recip data:

In [ ]:
filename = 'recip.zarr'
recip_DS = xr.open_zarr(zarrsPath.joinpath(filename)).compute()
recip_DS

### 2D reciprocal space images

In [ ]:
%matplotlib inline
plt.close('all')

# 2D reciprocal space cartesian plots
qxy_min = -1.1
qxy_max = 2.1
qz_min = 0
qz_max = 2.2

# Make & save or show plots
selected_attrs_dict = {}
selected_DAs = select_attrs(recip_DS.data_vars.values(), selected_attrs_dict)
        
for DA in tqdm(selected_DAs):
    # Slice data for selected q ranges (will need to rename q_xy if dimensions are differently named)
    sliced_DA = DA.sel(q_xy=slice(qxy_min, qxy_max), q_z=slice(qz_min, qz_max))

    # Feel free to mess around with the quantile selection...
    cmin = sliced_DA.quantile(0.3)
    cmax = sliced_DA.quantile(0.994) 
    
    # Plot
    ax = sliced_DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), interpolation='antialiased', figsize=(5.5,3.3))
    ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)
    ax.axes.set(aspect='equal', title=f'Cartesian Plot: {sn[DA.name]}',
                xlabel='q$_{xy}$ [Å$^{-1}$]', ylabel='q$_z$ [Å$^{-1}$]')
    ax.axes.xaxis.set_minor_locator(plt.MultipleLocator(0.1))
    ax.axes.yaxis.set_minor_locator(plt.MultipleLocator(0.1))
    ax.figure.set(tight_layout=True, dpi=130)
    
    # Save
    savePath = outPath.joinpath('recip_plots_v1')
    savePath.mkdir(exist_ok=True)
    ax.figure.savefig(savePath.joinpath(
        f'{DA.name}_qxy{qxy_min}to{qxy_max}_qz{qz_min}to{qz_max}_th{DA.incident_angle}.png'), dpi=150)

    plt.show()  # or show
    # plt.close('all')

## Polar plots

### Load dataset

In [ ]:
filename = 'caked.zarr'
raw_DS = xr.open_zarr(zarrsPath.joinpath(filename)).compute()
raw_DS

### Apply sin(chi), thickness, and other scalar corrections

In [ ]:
# # +/- 10 nm error
# thickness_error = 5
# thicknesses = {
#     'A1s':40,
#     'A1d':40,
#     'A2s':55,
#     'A2d':55,
#     'A3s':55,
#     'A3d':55
# }

In [ ]:
# Apply a sin chi correction
# Also add in thickness and/or exposure time corrections if applicable

sin_chi_DA = np.sin(np.radians(np.abs(raw_DS.chi)))

corr_DS = raw_DS.copy()
# corr_DS = corr_DS * sin_chi_DA  # This works mathematically, but does not preserve attributes
for i, var in enumerate(corr_DS.data_vars):
    corrected = corr_DS[var] * sin_chi_DA
    # corrected = corrected / thicknesses[var]  # how you could implement thickness correction with an iterable, dict would probably be better
    # corrected = corrected / (thicknesses[var] - thickness_error)  # how you could implement thickness correction with an iterable, dict would probably be better
    # corrected = corrected / (thicknesses[var] + thickness_error)  # how you could implement thickness correction with an iterable, dict would probably be better

    corr_DS[var].values = corrected.values
    
corr_DS

### 2D caked images

#### raw(chi) intensity polar plot

In [ ]:
# Polar plots, for raw(chi) intensities
DS = raw_DS.copy()

# Set chi range: Full range
chi_min = 0
chi_max = 90
q_min = 0.1
q_max = 2.2

# Select attribute
selected_attrs_dict = {}
selected_DAs = select_attrs(DS.data_vars.values(), selected_attrs_dict)
for DA in tqdm(selected_DAs):
    # Slice dataarray to select plotting region 
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min, q_max))
    
    # Set color limits
    cmin = sliced_DA.quantile(0.01)
    cmax = sliced_DA.quantile(0.995)
    
    # Plot sliced dataarray
    ax = sliced_DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), figsize=(5,4), interpolation='antialiased')  # plot, optional parameter interpolation='antialiased' for image smoothing

    # ax.axes.set(title=f'Polar plot: {DA.material} {DA.solvent} {DA.rpm}, $\\alpha_i$ = {float(DA.incident_angle[2:])}°, raw intensity')
    ax.axes.set(title=f'Polar plot: {sn[DA.film]}, $\\alpha_i$ = {DA.incident_angle}°, raw intensity')

    ax.colorbar.set_label('Raw intensity [arb. units]', rotation=270, labelpad=15)  # set colorbar label & parameters 
    ax.axes.set(xlabel='q$_r$ [Å$^{-1}$]', ylabel='$\\chi$ [°]')  # set title, axis labels, misc
    ax.figure.set(tight_layout=True, dpi=130)  # Adjust figure dpi & plotting style
    
    
    # # Save
    # savePath = outPath.joinpath('raw_caked_plots_v1')
    # savePath.mkdir(exist_ok=True)
    # ax.figure.savefig(savePath.joinpath(f'{DA.film}_{DA.film_number}_qr{q_min}to{q_max}_{DA.incident_angle}.png'), dpi=150)

    plt.show()  # Comment to mute plotting output
    plt.close('all')

#### sin(chi) intensity polar plot

In [ ]:
%matplotlib inline

In [ ]:
%matplotlib inline
plt.close('all')
# Polar plots, for sin(chi) intensities
DS = corr_DS.copy()

# Set chi range: Full range
chi_min = 0
chi_max = 90
q_min = 0.1
# q_min = 1
q_max = 2.2

# Select attribute
# selected_attrs_dict = {'film': ['A2s','A3s']}
selected_attrs_dict = {}
selected_DAs = select_attrs(DS.data_vars.values(), selected_attrs_dict)
for DA in tqdm(selected_DAs):
    # Slice dataarray to select plotting region 
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min, q_max))
    
    # Set color limits
    cmin = sliced_DA.sel(qr=slice(1.0,1.8)).quantile(0.01)
    # cmax = sliced_DA.quantile(0.995)  
    cmax = sliced_DA.sel(qr=slice(1.0,1.8)).quantile(0.9999)  
    
    # Plot sliced dataarray
    ax = sliced_DA.plot.imshow(origin='upper', cmap=cmap, norm=plt.Normalize(cmin, cmax), figsize=(5,4), interpolation='antialiased')  # plot, optional parameter interpolation='antialiased' for image smoothing
    ax.axes.set(title=f'Polar Plot: {sn[DA.name]}, sin($\\chi$) corr.')
    ax.colorbar.set_label('Intensity * sin($\\chi$) [arb. units]', rotation=270, labelpad=15)  # set colorbar label & parameters 
    ax.axes.set(xlabel='q$_r$ [Å$^{-1}$]', ylabel='$\\chi$ [°]')  # set title, axis labels, misc
    ax.figure.set(tight_layout=True, dpi=130)  # Adjust figure dpi & plotting style
        
    # Save
    savePath = outPath.joinpath('corr_caked_plots_v1')
    savePath.mkdir(exist_ok=True)
    ax.figure.savefig(savePath.joinpath(f'{DA.name}_qr{q_min}to{q_max}_{DA.incident_angle}_lower.png'), 
                      dpi=150)
    # ax.figure.savefig(savePath.joinpath(f'pipi_{DA.name}_qr{q_min}to{q_max}_{DA.incident_angle}.png'), 
    #                   dpi=150)
    
    plt.show()
    
    plt.close('all')

### 1D linecuts along chi

In [ ]:
chi_values

In [ ]:
chi_bins

In [ ]:
chi_values = np.linspace(chi_min, chi_max, 5, endpoint=True)
chi_values

In [ ]:
%matplotlib widget
plt.close('all')

# Polar plots, for sin(chi) intensities
DS = corr_DS.copy()

# Set chi bounds & bins, q bounds, etc.
# chi_min, chi_width, chi_bins = [14, 9.25, 8]  # for full chi cut
# chi_min, chi_width, chi_bins = [14, 4, 18]  # for full chi cut, precise wedges
# chi_min, chi_width, chi_bins = [14, 4, 6]  # for π-π chi cut
chi_min, chi_width, chi_bins = [14, 4, 2]  # for π-π chi cut
# chi_min, chi_width, chi_bins = [14, 2, 9]  # for precise π-π chi cut -> 38
# chi_min, chi_width, chi_bins = [14, 2, 3]  # for precise π-π chi cut -> 22

chi_max = chi_min + (chi_width * chi_bins)
colors = plt.cm.viridis_r(np.linspace(0.15,1,chi_bins))

# q_min = 0.13  # for full qr cut
q_min = 0.9  # for π-π qr cut
q_max = 2.07

# # For low-q peak
# q_min = 0.17
# q_max = 1

# Select attribute
# selected_attrs_dict = {'film': ['A2s','A3s']}
selected_attrs_dict = {}
selected_DAs = select_attrs(DS.data_vars.values(), selected_attrs_dict)

for DA in tqdm(selected_DAs):
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min,q_max))
    binned_DA = sliced_DA.groupby_bins('chi', chi_bins).sum('chi')

    max_val = float(binned_DA.sel(qr=slice(0.25, 1.6)).max())
    
    fig, ax = plt.subplots(figsize=(8,5), dpi=150, tight_layout=True)
    for i, chi_bin in enumerate(binned_DA.chi_bins.data[::-1]):
        lines = binned_DA.sel(chi_bins=chi_bin).plot.line(ax=ax, color=colors[i], label=chi_bin)
        mplcursors.cursor(lines)

    ax.set(title=f'{sn[DA.name]}: $\\chi$-binned-summed linecuts:\n'+ 
                 f'{chi_bins}, {chi_width}° $\\chi$ bins: from {int(chi_min)} to {int(chi_max)}°', 
           xlabel='$q_r$ $[Å^{-1}]$', 
           ylabel='$sin(\\chi) * Intensity$ [arb. units]',
           ylim=(None,max_val+(max_val*0.1)))
    ax.axes.xaxis.set_minor_locator(plt.MultipleLocator(0.1))
    ax.axes.grid(visible=True, which='both', axis='y')
    # ax.legend(title='Chi Bins')

    # Create a colormap and normalizer to match the plotted data
    sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=plt.Normalize(vmin=chi_min, vmax=chi_max))
    sm.set_array([])  # You need to set the array for the scalar mappable

    # Create the colorbar
    cbar = plt.colorbar(sm, ax=ax, orientation='vertical')
    cbar.set_label('Azimuthal Angle [°]', rotation=270, labelpad=15)
    
    # Define the ticks for the colorbar based on the chi bins
    chi_values = np.linspace(chi_min, chi_max, 5, endpoint=True)
    cbar.set_ticks(chi_values)
    cbar.set_ticklabels([f"{value:.1f}" for value in chi_values])

    # # Save
    # outPath.joinpath('chi-binned_linecuts_v1').mkdir(exist_ok=True)
    # # savePath = outPath.joinpath(
    # #     'chi-binned_linecuts_v1', f'full_chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
    # savePath = outPath.joinpath(
    #     'chi-binned_linecuts_v1', f'pipi_chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
    # savePath.mkdir(exist_ok=True)
    # ax.figure.savefig(savePath.joinpath(f'{DA.name}_qr{q_min}to{q_max}_{DA.incident_angle}.png'), 
    #                   dpi=150)
    
plt.show()
# plt.close('all')

### π-π line fitting*
Use lmfit to perform the linefits

#### π-π lmfit functions

In [ ]:
np.round((2*np.pi)/1.727, 2)

In [ ]:
np.round((2*np.pi)/1.738, 2)

In [ ]:
def pipi_lmfit(sliced_DA):
    """
    Function currently utilizes global variables, define in notebook!
    
    Inputs:
    - sliced_DA: 1D data for a selected chi bin, already meaned over chi
    """    
    point_y = float(sliced_DA.sel(qr=slice(0.9,1)).mean('qr'))
    point_y = point_y - point_y*(0.15)

    x = sliced_DA.qr.data
    y = sliced_DA.data

    # Define all models to include in fitting
    bkg_mod = models.LinearModel(prefix='bkg_')
    pars = bkg_mod.make_params(intercept=point_y, slope=0)
    pars['bkg_intercept'].set(vary=False)
    pars['bkg_slope'].set(vary=False)

    pipi_mod = models.PseudoVoigtModel(prefix='pipi_')  # pi-pi peak
    pars += pipi_mod.guess(y, x, center=1.72)
    pars['pipi_amplitude'].set(min=0)
    pars['pipi_center'].set(min=1.7, max=1.78)
    # pars['pipi_center'].set(min=1.68, max=1.76)
    pars['pipi_sigma'].set(min=0.15, max=0.23)
    
    sidechains_mod = models.PseudoVoigtModel(prefix='sidechains_')  # sidechains peak for acceptor films
    pars += sidechains_mod.guess(y, x, center=1.5)
    pars['sidechains_amplitude'].set(min=0)
    pars['sidechains_center'].set(min=1.45, max=1.55)
    pars['sidechains_sigma'].set(min=0.18, max=0.28)
    
    PSpk2_mod = models.PseudoVoigtModel(prefix='PSpk2_')  # PS second peak around 1.37
    pars += PSpk2_mod.guess(y, x, center=1.37)
    pars['PSpk2_amplitude'].set(min=0)
    pars['PSpk2_center'].set(min=1.3, max=1.44)
    pars['PSpk2_sigma'].set(min=0.18, max=0.28)


    # Combine into full model
    # mod = bkg_mod + pipi_mod + sidechains_mod + PSpk2_mod
    mod = bkg_mod + pipi_mod + PSpk2_mod

    # Run fit and store all info in a ModelResult object
    out = mod.fit(y, pars, x=x)
    return out

def pipi_fit(sliced_DA):   
    # Run lmfit
    out = pipi_lmfit(sliced_DA)
    FWHM = np.round(float(out.params['pipi_fwhm']), 2)
    Lc = np.round((2*np.pi*0.9)/float(out.params['pipi_fwhm']), 2)
    center = np.round(float(out.params['pipi_center']), 2)
    dspacing = np.round((2*np.pi)/float(out.params['pipi_center']), 2)
    
    amplitude = np.round(float(out.params['pipi_amplitude']), 3)

    chi_range = f'{str(round(sliced_DA.chi_bin.left))}-{str(round(sliced_DA.chi_bin.right))}'
    
    # rsquared = np.round(out.rsquared, 3)
    redchi = np.round(out.redchi, 5)

    AA1 = '$\\AA^{-1}$'
    
    # Plot
    q = sliced_DA.qr.data
    I = sliced_DA.data
    
    fig, ax = plt.subplots()
    fig.set(size_inches=(6.5,3.5), dpi=120, tight_layout=True)
       
    ax.plot(q, I, label='data', linewidth=2.5)
    ax.plot(q, out.best_fit, '--', label='full_fit')
    for key in out.eval_components():
        ax.plot(q, out.eval_components()[key], label=f'{key}')
    ax.set(xlabel=f'Q [{AA1}]', ylabel='Intensity [arb. units]', yscale='linear')
    ax.set_title(
        f'π-π peak fit: {sn[sliced_DA.name]}, {chi_range}° chi; peak area = {amplitude}\n' + 
        f'center = {center}, FWHM = {FWHM} {AA1} (d-spacing = {dspacing}, $L_c$ = {Lc} $\\AA$)', 
        x=0.6)
    ax.legend(title='$\\chi_{red}$ '+f'= {redchi}', loc='upper left', bbox_to_anchor=(1,1))
    ax.grid(visible=True, axis='x')
    ax.xaxis.set_minor_locator(plt.MultipleLocator(0.1))
    
    return out, fig, ax

#### Run π-π fits

In [ ]:
plt.close('all')

In [ ]:
%matplotlib inline

In [ ]:
# Peak fitting for each chi bin
DS = corr_DS.copy()

# Set chi bounds & bins, q bounds, etc.
# chi_min, chi_width, chi_bins = [14, 9.25, 8]  # for full chi cut
# chi_min, chi_width, chi_bins = [14, 4, 18]  # for full chi cut, precise wedges
# chi_min, chi_width, chi_bins = [14, 4, 6]  # for π-π chi cut -> 38
chi_min, chi_width, chi_bins = [14, 2, 9]  # for precise π-π chi cut -> 38
# chi_min, chi_width, chi_bins = [14, 4, 7]  # for π-π chi cut -> 42
chi_max = chi_min + (chi_width * chi_bins)
colors = plt.cm.viridis_r(np.linspace(0.15,1,chi_bins))

# q_min = 0.1  # for full qr cut
q_min = 0.9  # for π-π qr cut
q_max = 2.07

# Select attributes
# selected_attrs_dict = {'film': ['A2s', 'A2d', 'A3s', 'A3d']}
selected_attrs_dict = {}
selected_DAs = select_attrs(DS.data_vars.values(), selected_attrs_dict)

# For each selected DA, fit the π-π peak
all_outs = {}
all_params = {}
for DA in tqdm(selected_DAs):
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min,q_max))
    binned_DA = sliced_DA.groupby_bins('chi', chi_bins).sum('chi')
    outs = {}
    params = {}
    for i, chi_bin in enumerate(binned_DA.chi_bins.data[::-1]):
        key = f'{str(round(chi_bin.left))}-{str(round(chi_bin.right))}'
        sel_DA = binned_DA.sel(chi_bins=chi_bin)
        sel_DA.attrs['chi_bin'] = chi_bin
        outs[key], fig, ax = pipi_fit(sel_DA)
        params[key] = outs[key].params.valuesdict()

        # Save
        outPath.joinpath('pi-pi_linefits_v1').mkdir(exist_ok=True)
        savePath = outPath.joinpath('pi-pi_linefits_v1', f'chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
        savePath.mkdir(exist_ok=True)
        fig.savefig(savePath.joinpath(f'{DA.name}_chi{key}_pi-pi_fit.png'), dpi=150)

        # plt.show()
        plt.close('all')
        
    all_outs[DA.name] = outs
    all_params[DA.name] = params

# Save model_params.json in same save folder
json_data = json.dumps(all_params)
with open(str(savePath.joinpath('model_params.json')), 'w') as f:
    f.write(json_data)

# Create full fit reports folder and save entire report .txt files inside:
savePath.joinpath('fit_reports').mkdir(exist_ok=True)
for film, outs in all_outs.items():
    for chi_bin, out in outs.items():
        with savePath.joinpath('fit_reports', f'{film}_{chi_bin}_fit_result.txt').open(mode='w') as f:
            f.write(out.fit_report())

In [ ]:
pairs = [('A1s', 'A1d'), ('A2s', 'A2d'), ('A3s', 'A3d')]
avg_params, avg_param_errors = average_multiple_pairs(all_params, pairs, ['A1', 'A2', 'A3'])

In [ ]:
import matplotlib.ticker as ticker

In [ ]:
colors

In [ ]:
all_params

In [ ]:
# Plot pseudovoigt fractions, d-spacings, scherrer coherence lengths, and normalized peak areas vs chi

# Initialize figures to be populated later
# fig_psvoigt, ax_psvoigt = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
fig_dspacing, ax_dspacing = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
fig_ccl, ax_ccl = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
fig_pole, ax_pole = plt.subplots(figsize=((2.8,3.3)), dpi=150, tight_layout=True)

# # Define custom colors for each film, here I've chosen just two shades along the same sequention colorbar for each sample group
# colors_dict = {
#     'A1s':plt.cm.Blues(np.linspace(0.6,0.8,2))[0],
#     'A1d':plt.cm.Blues(np.linspace(0.6,0.8,2))[1],
#     'A2s':plt.cm.Greens(np.linspace(0.6,0.8,2))[0],
#     'A2d':plt.cm.Greens(np.linspace(0.6,0.8,2))[1],
#     'A3s':plt.cm.Oranges(np.linspace(0.6,0.8,2))[0],
#     'A3d':plt.cm.Oranges(np.linspace(0.6,0.8,2))[1]
# }

# colors_dict = {
#     'A1': (247/255, 150/255,  69/255),
#     'A2': (191/255, 194/255, 194/255),
#     'A3': ( 48/255, 134/255, 155/255),
# }

colors = plt.cm.viridis_r(np.linspace(0.05,1,3))
# colors = plt.cm.YlGnBu(np.linspace(0.3,1,3))
colors_dict = {
    'A1': colors[0],
    'A2': colors[1],
    'A3': colors[2],
}

# colors_dict = {
#     'A1': '#edf8b1',
#     'A2': '#7fcdbb',
#     'A3': '#2c7fb8',
# }

### Extracting info from all_params & populating ___ vs chi plots:
summed_peak_areas = {}
avg_dspacings = {}
avg_coherence_lengths = {}
for film in avg_params:
    peak_areas = []
    pvoigt_fracs = []
    peak_centers = []
    peak_fwhms = []
    
    peak_area_errors = []
    peak_center_errors = []
    peak_fwhm_errors = []
    
    chis = []
    for chi_bin, params, errors in zip(avg_params[film].keys(), avg_params[film].values(), avg_param_errors[film].values()):
        # Get starting value of chi bin
        chi = int(chi_bin.split('-')[0]) + 1
        chis.append(chi)
        # Get peak areas
        peak_area = params['pipi_amplitude']
        peak_areas.append(peak_area)
        peak_area_error = errors['pipi_amplitude']
        peak_area_errors.append(peak_area_error)
        # Get pseudovoigt fraction
        pvoigt_frac = params['pipi_fraction']
        pvoigt_fracs.append(pvoigt_frac)
        # Get peak centers
        peak_center = params['pipi_center']
        peak_centers.append(peak_center)
        peak_center_error = errors['pipi_center']
        peak_center_errors.append(peak_center_error)
        # Get peak fwhms
        peak_fwhm = params['pipi_fwhm']
        peak_fwhms.append(peak_fwhm)
        peak_fwhm_error = errors['pipi_fwhm']
        peak_fwhm_errors.append(peak_fwhm_error)

    # Get normalized peak areas (divide by sum of all peak areas; adds to 1) and write area sum to dict
    peak_areas = np.array(peak_areas)
    summed_peak_areas[film] = np.sum(peak_areas)  # Write sum of peak area out, this corresponds to relative extent of crystallinity if thickness normalized
    normed_peak_areas = peak_areas / np.sum(peak_areas)
    normed_peak_area_errors = normed_peak_areas * np.array(peak_area_errors)/np.array(peak_areas)

    # Calculate d-spacings from peak centers, calculate peak-area-weighted average of d-spacing
    dspacings = (2*np.pi) / np.array(peak_centers) 
    dspacing_errors = dspacings * np.array(peak_center_errors)/np.array(peak_centers)
    
    avg_dspacing = np.round(np.sum(dspacings*normed_peak_areas), 2)
    avg_dspacings[film] = avg_dspacing  

    # Calculate coherence length from peak fwhm, calculate peak-area-weighted average of coherence length    
    coherence_lengths = (2*np.pi*0.9) / np.array(peak_fwhms) 
    # coherence_length_errors = (2*np.pi*0.9) / np.array(peak_fwhm_errors) 
    coherence_length_errors = coherence_lengths * np.array(peak_fwhm_errors)/np.array(peak_fwhms)

    avg_coherence_length = np.round(np.sum(coherence_lengths*normed_peak_areas), 2)
    avg_coherence_lengths[film] = avg_coherence_length

    ### Plotting
    # # Plot pseudovoigt fraction vs chi for each film
    # ax_psvoigt.errorbar(chis, pvoigt_fracs, label=film, color=colors_dict[film], marker='o')
    # ax_psvoigt.set(title='Pseudo-voigt lorentzian-to-gaussian ratio versus $\\chi$',
    #                ylabel='Lorentzian peak fraction',
    #                xlabel='Binned $\\chi$ value [°]')
    # ax_psvoigt.legend(title='Film', loc='upper left', bbox_to_anchor=(1,1))

    # Plot d-spacing vs chi for each film
    ax_dspacing.errorbar(chis, dspacings, label=film, color=colors_dict[film], marker='o', 
                         yerr=dspacing_errors+0.015, capsize=4)
    ax_dspacing.set(title='D-spacing versus $\\chi$',
                    ylabel='d-spacing [Å]',
                    xlabel='$\\chi$ value [°]',
                    ylim=(3.55,3.8))
    ax_dspacing.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.85))
    ax_dspacing.grid(visible=True,which='major',axis='y')
    # ax_dspacing.legend(title='Film')

    # Plot coherence length vs chi for each film
    ax_ccl.errorbar(chis, coherence_lengths, label=film, color=colors_dict[film], marker='o', 
                    yerr=coherence_length_errors+0.3, capsize=4)
    ax_ccl.set(title='Crystalline coherence length (CCL) versus $\\chi$',
               ylabel='CCL [Å]',
               xlabel='$\\chi$ value [°]',
               ylim=(13,18.5))
    ax_ccl.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.85))
    ax_ccl.grid(visible=True,which='major',axis='y')
    # ax_ccl.yaxis.set_major_locator(ticker.MultipleLocator(0.05))

    # ax_ccl.legend(title='Film')
    
    # Plot 'pole figure' for each film
    ax_pole.errorbar(chis, normed_peak_areas, label=film, color=colors_dict[film], marker='o',
                     yerr=normed_peak_area_errors+0.005, capsize=4)
    ax_pole.set(title='π-π peak pole figure',
                ylabel='Peak area [arb. units]',
                xlabel='$\\chi$ value [°]')
                # ylim=(0.08,0.26))
    ax_pole.set_ybound(lower=0, upper=0.21)
    # ax_pole.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.85))  
    ax_pole.grid(visible=True,which='major',axis='y')
    # ax_pole.set_yticks
    # ax_pole.yaxis.set_major_locator(ticker.MultipleLocator(0.05))
    ax_pole.legend(title='Film')    

    # Extract chi values for file saving purposes
    chi_width = chis[0]-chis[1]
    chi_bins = len(chis)
    chi_min = chis[-1] - 1
    chi_max = chis[0]+chi_width - 1

    # # Save
    # parent_folder = 'pi-pi_fit_results_plots_v3'
    # outPath.joinpath(parent_folder).mkdir(exist_ok=True)
    # savePath = outPath.joinpath(parent_folder, f'chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
    # savePath.mkdir(exist_ok=True)
    # # fig_psvoigt.savefig(savePath.joinpath(f'pseudovoigt_ratios_pi-pi_fit.png'), dpi=150)
    # # fig_dspacing.savefig(savePath.joinpath(f'dspacings_pi-pi_fit.png'), dpi=150)
    # # fig_ccl.savefig(savePath.joinpath(f'coherence_lengths_pi-pi_fit.png'), dpi=150)
    # # fig_pole.savefig(savePath.joinpath(f'peak_areas_pi-pi_fit.png'), dpi=150)
    # fig_pole.savefig(savePath.joinpath(f'peak_areas_pi-pi_fit_narrow.png'), dpi=150)
    
plt.show()
plt.close('all')  

In [ ]:
outPath

#### Plot fit results

In [ ]:
# all_outs['A1d']['14-16']

In [ ]:
# good sigma = 	0.19014368
# bad sigma = 	0.45324640, 0.028 # way too sharp

In [ ]:
# Load params from json file
# chi_width = 4
# chi_bins = 6
# chi_range = '14-38'
# chi_width = 4
# chi_bins = 7
# chi_range = '14-42'
# chi_width = 4
# chi_bins = 18
# chi_range = '14-86'

with open(str(outPath.joinpath('pi-pi_linefits_v3', f'chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_range}', 'model_params.json')), 'r') as f:
    all_params = json.load(f)

len(all_params)

In [ ]:
all_params

In [ ]:
def average_multiple_pairs(data, pairs, outkeys):  
    all_params = {}
    all_param_errors = {}
    for pair, outkey in zip(pairs, outkeys):
        key1, key2 = pair
        averaged_results = {}
        differences_results = {}
    
        # Get the sub-dictionaries for both keys
        dict1 = data[key1]
        dict2 = data[key2]
        
        # Iterate over the keys in the first dictionary
        for key in dict1:
            if key in dict2:
                # Initialize sub-dictionaries for averages and differences
                averaged_results[key] = {}
                differences_results[key] = {}
                
                # Iterate over the nested keys and values
                for subkey in dict1[key]:
                    if subkey in dict2[key]:
                        val1 = dict1[key][subkey]
                        val2 = dict2[key][subkey]
                        
                        # Calculate average and difference
                        average = (val1 + val2) / 2
                        difference = abs(val1 - val2) / 2
                        
                        # Store the results
                        averaged_results[key][subkey] = average
                        differences_results[key][subkey] = difference

        all_params[outkey] = averaged_results
        all_param_errors[outkey] = differences_results
        
    return all_params, all_param_errors

In [ ]:
pairs = [('A1s', 'A1d'), ('A2s', 'A2d'), ('A3s', 'A3d')]
avg_params, avg_param_errors = average_multiple_pairs(all_params, pairs, ['A1', 'A2', 'A3'])

In [ ]:
avg_params

In [ ]:
avg_param_errors

In [ ]:
list(all_params.keys())

In [ ]:
# selected_samples = ['PSs_01_0_00000', 'PStoA1s_1to1_0_00000', 'PStoA1s_3to1_0_00000', 'PStoA1s_9to1_0_00000', 'PStoA2s_1to1_0_00000', 'PStoA2s_3to1_0_00000', 'PStoA2s_9to1_0_00000', 
#                     'PStoA3s_1to1_0_00000', 'PStoA3s_3to1_0_00000', 'PStoA3s_9to1_0_00000']

# selected_samples = ['PStoA1s_1to1_0_00000', 'PStoA1s_3to1_0_00000', 'PStoA1s_9to1_0_00000', 'PStoA2s_1to1_0_00000', 'PStoA2s_3to1_0_00000', 'PStoA2s_9to1_0_00000', 
#                     'PStoA3s_1to1_0_00000', 'PStoA3s_3to1_0_00000', 'PStoA3s_9to1_0_00000']

# selected_samples = ['PStoA1s_1to1_0_00000', 'PStoA1s_3to1_0_00000', 'PStoA2s_1to1_0_00000', 'PStoA2s_3to1_0_00000', 
#                     'PStoA3s_1to1_0_00000', 'PStoA3s_3to1_0_00000']

# selected_samples = ['PStoA1s_1to1_0_00000', 'PStoA2s_1to1_0_00000', 'PStoA2s_3to1_0_00000', 
#                     'PStoA3s_1to1_0_00000', 'PStoA3s_3to1_0_00000']

selected_samples = ['PStoA1s_1to1_0_00000', 'PStoA2s_1to1_0_00000', 'PStoA3s_1to1_0_00000']

In [ ]:
# Plot pseudovoigt fractions, d-spacings, scherrer coherence lengths, and normalized peak areas vs chi
# Halfway to updating to use just averaged pairs and error bars

# Initialize figures to be populated later
# fig_psvoigt, ax_psvoigt = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
fig_dspacing, ax_dspacing = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
fig_ccl, ax_ccl = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
fig_pole, ax_pole = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)

# # Define custom colors for each film, here I've chosen just two shades along the same sequention colorbar for each sample group
# colors_dict = {
#     'A1s':plt.cm.Blues(np.linspace(0.6,0.8,2))[0],
#     'A1d':plt.cm.Blues(np.linspace(0.6,0.8,2))[1],
#     'A2s':plt.cm.Greens(np.linspace(0.6,0.8,2))[0],
#     'A2d':plt.cm.Greens(np.linspace(0.6,0.8,2))[1],
#     'A3s':plt.cm.Oranges(np.linspace(0.6,0.8,2))[0],
#     'A3d':plt.cm.Oranges(np.linspace(0.6,0.8,2))[1]
# }
colors = plt.cm.viridis_r(np.linspace(0.05,1,3))
colors_dict = {
    'PStoA1s_1to1_0_00000': colors[0],
    'PStoA2s_1to1_0_00000': colors[1],
    'PStoA3s_1to1_0_00000': colors[2],
    'PStoA1s_3to1_0_00000': np.append((colors[0][:3]*0.8), 1),
    'PStoA2s_3to1_0_00000': np.append((colors[1][:3]*0.8), 1),
    'PStoA3s_3to1_0_00000': np.append((colors[2][:3]*0.8), 1),
    'PStoA1s_9to1_0_00000': np.append((colors[0][:3]*0.6), 1),
    'PStoA2s_9to1_0_00000': np.append((colors[1][:3]*0.6), 1),
    'PStoA3s_9to1_0_00000': np.append((colors[2][:3]*0.6), 1),
}

### Extracting info from all_params & populating ___ vs chi plots:
summed_peak_areas = {}
avg_dspacings = {}
avg_coherence_lengths = {}
for film in selected_samples:
    peak_areas = []
    pvoigt_fracs = []
    peak_centers = []
    peak_fwhms = []
    
    chis = []
    for chi_bin, params in all_params[film].items():
        # Get starting value of chi bin
        chi = int(chi_bin.split('-')[0])
        chis.append(chi)
        # Get peak areas
        peak_area = params['pipi_amplitude']
        peak_areas.append(peak_area)
        # Get pseudovoigt fraction
        pvoigt_frac = params['pipi_fraction']
        pvoigt_fracs.append(pvoigt_frac)
        # Get peak centers
        peak_center = params['pipi_center']
        peak_centers.append(peak_center)
        # Get peak fwhms
        peak_fwhm = params['pipi_fwhm']
        peak_fwhms.append(peak_fwhm)
        
    # Get normalized peak areas (divide by sum of all peak areas; adds to 1) and write area sum to dict
    peak_areas = np.array(peak_areas)
    summed_peak_areas[film] = np.sum(peak_areas)  # Write sum of peak area out, this corresponds to relative extent of crystallinity if thickness normalized
    normed_peak_areas = peak_areas / np.sum(peak_areas)

    # Calculate d-spacings from peak centers, calculate peak-area-weighted average of d-spacing
    dspacings = (2*np.pi) / np.array(peak_centers) 
    avg_dspacing = np.round(np.sum(dspacings*normed_peak_areas), 2)
    avg_dspacings[film] = avg_dspacing  

    # Calculate coherence length from peak fwhm, calculate peak-area-weighted average of coherence length    
    coherence_lengths = (2*np.pi*0.9) / np.array(peak_fwhms) 
    avg_coherence_length = np.round(np.sum(coherence_lengths*normed_peak_areas), 2)
    avg_coherence_lengths[film] = avg_coherence_length

    ### Plotting
    # # Plot pseudovoigt fraction vs chi for each film
    # ax_psvoigt.errorbar(chis, pvoigt_fracs, label=film, color=colors_dict[film], marker='o')
    # ax_psvoigt.set(title='Pseudo-voigt lorentzian-to-gaussian ratio versus $\\chi$',
    #                ylabel='Lorentzian peak fraction',
    #                xlabel='Binned $\\chi$ value [°]')
    # ax_psvoigt.legend(title='Film', loc='upper left', bbox_to_anchor=(1,1))

    # Plot d-spacing vs chi for each film
    ax_dspacing.errorbar(chis, dspacings, label=sn[film], color=colors_dict[film], marker='o')
    ax_dspacing.set(title='D-spacing versus $\\chi$',
                    ylabel='d-spacing [Å]',
                    xlabel='Binned $\\chi$ value [°]')
                    # ylim=(3.6,3.9))
    ax_dspacing.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.85))

    # Plot coherence length vs chi for each film
    ax_ccl.errorbar(chis, coherence_lengths, label=sn[film], color=colors_dict[film], marker='o')
    ax_ccl.set(title='Crystalline coherence length (CCL) versus $\\chi$',
               ylabel='CCL [Å]',
               xlabel='Binned $\\chi$ value [°]')
               # ylim=(12,18))
    ax_ccl.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.85))
    
    # Plot 'pole figure' for each film
    ax_pole.errorbar(chis, normed_peak_areas, label=sn[film], color=colors_dict[film], marker='o')
    ax_pole.set(title='Normalized π-π peak areas versus $\\chi$:\n'+
                 'peak area divided by sum of all peak areas ',
                ylabel='Peak area [arb. units]',
                xlabel='Binned $\\chi$ value [°]')
                # ylim=(0.08,0.26))
    # ax_pole.set_ybound(lower=0)
    ax_pole.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.87))    

    # Extract chi values for file saving purposes
    chi_width = chis[0]-chis[1]
    chi_bins = len(chis)
    chi_min = chis[-1]
    chi_max = chis[0]+chi_width

    # # Save
    # parent_folder = 'pi-pi_fit_results_plots_v2'
    # outPath.joinpath(parent_folder).mkdir(exist_ok=True)
    # savePath = outPath.joinpath(parent_folder, f'chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
    # savePath.mkdir(exist_ok=True)
    # # fig_psvoigt.savefig(savePath.joinpath(f'pseudovoigt_ratios_pi-pi_fit.png'), dpi=150)
    # fig_dspacing.savefig(savePath.joinpath(f'dspacings_pi-pi_fit.png'), dpi=150)
    # fig_ccl.savefig(savePath.joinpath(f'coherence_lengths_pi-pi_fit.png'), dpi=150)
    # fig_pole.savefig(savePath.joinpath(f'peak_areas_pi-pi_fit.png'), dpi=150)
    
plt.show()
plt.close('all')  

In [ ]:
peak_ar

In [ ]:
# Plot pseudovoigt fractions, d-spacings, scherrer coherence lengths, and normalized peak areas vs chi

# Initialize figures to be populated later
# fig_psvoigt, ax_psvoigt = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
fig_dspacing, ax_dspacing = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
fig_ccl, ax_ccl = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
fig_pole, ax_pole = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)

# # Define custom colors for each film, here I've chosen just two shades along the same sequention colorbar for each sample group
# colors_dict = {
#     'A1s':plt.cm.Blues(np.linspace(0.6,0.8,2))[0],
#     'A1d':plt.cm.Blues(np.linspace(0.6,0.8,2))[1],
#     'A2s':plt.cm.Greens(np.linspace(0.6,0.8,2))[0],
#     'A2d':plt.cm.Greens(np.linspace(0.6,0.8,2))[1],
#     'A3s':plt.cm.Oranges(np.linspace(0.6,0.8,2))[0],
#     'A3d':plt.cm.Oranges(np.linspace(0.6,0.8,2))[1]
# }

colors_dict = {
    'A1': (247/255, 150/255,  69/255),
    'A2': (191/255, 194/255, 194/255),
    'A3': ( 48/255, 134/255, 155/255),
}

### Extracting info from all_params & populating ___ vs chi plots:
summed_peak_areas = {}
avg_dspacings = {}
avg_coherence_lengths = {}
for film in avg_params:
    peak_areas = []
    pvoigt_fracs = []
    peak_centers = []
    peak_fwhms = []
    
    peak_area_errors = []
    peak_center_errors = []
    peak_fwhm_errors = []
    
    chis = []
    for chi_bin, params, errors in zip(avg_params[film].keys(), avg_params[film].values(), avg_param_errors[film].values()):
        # Get starting value of chi bin
        chi = int(chi_bin.split('-')[0])
        chis.append(chi)
        # Get peak areas
        peak_area = params['pipi_amplitude']
        peak_areas.append(peak_area)
        peak_area_error = errors['pipi_amplitude']
        peak_area_errors.append(peak_area_error)
        # Get pseudovoigt fraction
        pvoigt_frac = params['pipi_fraction']
        pvoigt_fracs.append(pvoigt_frac)
        # Get peak centers
        peak_center = params['pipi_center']
        peak_centers.append(peak_center)
        peak_center_error = errors['pipi_center']
        peak_center_errors.append(peak_center_error)
        # Get peak fwhms
        peak_fwhm = params['pipi_fwhm']
        peak_fwhms.append(peak_fwhm)
        peak_fwhm_error = errors['pipi_fwhm']
        peak_fwhm_errors.append(peak_fwhm_error)
        
    # Get normalized peak areas (divide by sum of all peak areas; adds to 1) and write area sum to dict
    peak_areas = np.array(peak_areas)
    summed_peak_areas[film] = np.sum(peak_areas)  # Write sum of peak area out, this corresponds to relative extent of crystallinity if thickness normalized
    normed_peak_areas = peak_areas / np.sum(peak_areas)
    normed_peak_area_errors = normed_peak_areas * np.array(peak_area_errors)/np.array(peak_areas)

    # Calculate d-spacings from peak centers, calculate peak-area-weighted average of d-spacing
    dspacings = (2*np.pi) / np.array(peak_centers) 
    dspacing_errors = dspacings * np.array(peak_center_errors)/np.array(peak_centers)
    
    avg_dspacing = np.round(np.sum(dspacings*normed_peak_areas), 2)
    avg_dspacings[film] = avg_dspacing  

    # Calculate coherence length from peak fwhm, calculate peak-area-weighted average of coherence length    
    coherence_lengths = (2*np.pi*0.9) / np.array(peak_fwhms) 
    # coherence_length_errors = (2*np.pi*0.9) / np.array(peak_fwhm_errors) 
    coherence_length_errors = coherence_lengths * np.array(peak_fwhm_errors)/np.array(peak_fwhms)

    avg_coherence_length = np.round(np.sum(coherence_lengths*normed_peak_areas), 2)
    avg_coherence_lengths[film] = avg_coherence_length

    ### Plotting
    # # Plot pseudovoigt fraction vs chi for each film
    # ax_psvoigt.errorbar(chis, pvoigt_fracs, label=film, color=colors_dict[film], marker='o')
    # ax_psvoigt.set(title='Pseudo-voigt lorentzian-to-gaussian ratio versus $\\chi$',
    #                ylabel='Lorentzian peak fraction',
    #                xlabel='Binned $\\chi$ value [°]')
    # ax_psvoigt.legend(title='Film', loc='upper left', bbox_to_anchor=(1,1))

    # Plot d-spacing vs chi for each film
    ax_dspacing.errorbar(chis, dspacings, label=film, color=colors_dict[film], marker='o', yerr=dspacing_errors, capsize=4)
    ax_dspacing.set(title='D-spacing versus $\\chi$',
                    ylabel='d-spacing [Å]',
                    xlabel='Binned $\\chi$ value [°]',
                    ylim=(3.45,3.85))
    ax_dspacing.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.85))
    # ax_dspacing.legend(title='Film')

    # Plot coherence length vs chi for each film
    ax_ccl.errorbar(chis, coherence_lengths, label=film, color=colors_dict[film], marker='o', 
                    yerr=coherence_length_errors, capsize=4)
    ax_ccl.set(title='Crystalline coherence length (CCL) versus $\\chi$',
               ylabel='CCL [Å]',
               xlabel='Binned $\\chi$ value [°]',
               ylim=(12,19))
    ax_ccl.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.85))
    # ax_ccl.legend(title='Film')
    
    # Plot 'pole figure' for each film
    ax_pole.errorbar(chis, normed_peak_areas, label=film, color=colors_dict[film], marker='o',
                     yerr=normed_peak_area_errors, capsize=4)
    ax_pole.set(title='Normalized π-π peak areas versus $\\chi$',
                ylabel='Peak area [arb. units]',
                xlabel='Binned $\\chi$ value [°]')
                # ylim=(0.08,0.26))
    ax_pole.set_ybound(lower=0)
    ax_pole.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.85))    
    # ax_pole.legend(title='Film')    

    # Extract chi values for file saving purposes
    chi_width = chis[0]-chis[1]
    chi_bins = len(chis)
    chi_min = chis[-1]
    chi_max = chis[0]+chi_width

    # # Save
    # parent_folder = 'pi-pi_fit_results_plots_v1'
    # outPath.joinpath(parent_folder).mkdir(exist_ok=True)
    # savePath = outPath.joinpath(parent_folder, f'chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
    # savePath.mkdir(exist_ok=True)
    # fig_psvoigt.savefig(savePath.joinpath(f'pseudovoigt_ratios_pi-pi_fit.png'), dpi=150)
    # fig_dspacing.savefig(savePath.joinpath(f'dspacings_pi-pi_fit.png'), dpi=150)
    # fig_ccl.savefig(savePath.joinpath(f'coherence_lengths_pi-pi_fit.png'), dpi=150)
    # fig_pole.savefig(savePath.joinpath(f'peak_areas_pi-pi_fit.png'), dpi=150)
    
plt.show()
plt.close('all')  

In [ ]:
coherence_lengths * np.array(peak_fwhm_errors)/np.array(peak_fwhms)

In [ ]:
np.array(peak_fwhm_errors)/np.array(peak_fwhms)

In [ ]:
avg_dspacings

In [ ]:
avg_coherence_lengths

In [ ]:
# Plot average d-spacing, scherrer coherence length, summed peak area vs film
fig_dspacing, ax_dspacing = plt.subplots(figsize=((2.5,2.5)), dpi=150, tight_layout=True)
fig_ccl, ax_ccl = plt.subplots(figsize=((2.5,2.5)), dpi=150, tight_layout=True)
# fig_area, ax_area = plt.subplots(figsize=((2.5,2.5)), dpi=150, tight_layout=True)

colors = list(colors_dict.values())
ax_dspacing.errorbar(list(avg_dspacings.keys()), list(avg_dspacings.values()), fmt='s')
ax_ccl.errorbar(list(avg_coherence_lengths.keys()), list(avg_coherence_lengths.values()), fmt='s')
# ax_area.errorbar(list(summed_peak_areas.keys()), list(summed_peak_areas.values()), fmt='s', yerr=errors)

ax_dspacing.set(title='Peak-area-weighted average d-spacings', xlabel='Film', ylabel='D-spacing [Å]')
ax_ccl.set(title='Peak-area-weighted average coherence lengths', xlabel='Film', ylabel='CCL [Å]')
# ax_area.set(title='Summed peak areas (error bars from thickness range)', xlabel='Film', ylabel='Intensity [arb. units]')

for ax in [ax_ccl, ax_dspacing]:
    ax.grid(visible=True, axis='y')

# # Save
# parent_folder = 'pi-pi_fit_results_plots_v1'
# outPath.joinpath(parent_folder).mkdir(exist_ok=True)
# savePath = outPath.joinpath(parent_folder, f'chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
# savePath.mkdir(exist_ok=True)
# fig_dspacing.savefig(savePath.joinpath(f'avg_dspacings_pi-pi_fit.png'), dpi=150)
# fig_ccl.savefig(savePath.joinpath(f'avg_coherence_lengths_pi-pi_fit.png'), dpi=150)
# fig_area.savefig(savePath.joinpath(f'summed_area_pi-pi_fit.png'), dpi=150)

plt.show()
plt.close('all')
display(avg_dspacings)
display(avg_coherence_lengths)
display(summed_peak_areas)

In [ ]:
# Average dspacing for each of two duplicate measurements
sd_avg_dspacings = {
    'A1': (avg_dspacings['A1d'] + avg_dspacings['A1s'])/2,
    'A2': (avg_dspacings['A2d'] + avg_dspacings['A2s'])/2,
    'A3': (avg_dspacings['A3d'] + avg_dspacings['A3s'])/2 
}

sd_avg_coherence_lengths = {
    'A1': (avg_coherence_lengths['A1d'] + avg_coherence_lengths['A1s'])/2,
    'A2': (avg_coherence_lengths['A2d'] + avg_coherence_lengths['A2s'])/2,
    'A3': (avg_coherence_lengths['A3d'] + avg_coherence_lengths['A3s'])/2 
}

sd_summed_peak_areas = {
    'A1': (summed_peak_areas['A1d'] + summed_peak_areas['A1s'])/2,
    'A2': (summed_peak_areas['A2d'] + summed_peak_areas['A2s'])/2,
    'A3': (summed_peak_areas['A3d'] + summed_peak_areas['A3s'])/2 
}

sd_dspacing_errors = {
    'A1': [sd_avg_dspacings['A1'] - min(avg_dspacings['A1d'], avg_dspacings['A1s']), 
           max(avg_dspacings['A1d'], avg_dspacings['A1s']) - sd_avg_dspacings['A1']],
    'A2': [sd_avg_dspacings['A2'] - min(avg_dspacings['A2d'], avg_dspacings['A2s']), 
           max(avg_dspacings['A2d'], avg_dspacings['A2s']) - sd_avg_dspacings['A2']],
    'A3': [sd_avg_dspacings['A3'] - min(avg_dspacings['A3d'], avg_dspacings['A3s']), 
           max(avg_dspacings['A3d'], avg_dspacings['A3s']) - sd_avg_dspacings['A3']]
}

sd_coherence_length_errors = {
    'A1': [sd_avg_coherence_lengths['A1'] - min(avg_coherence_lengths['A1d'], avg_coherence_lengths['A1s']), 
           max(avg_coherence_lengths['A1d'], avg_coherence_lengths['A1s']) - sd_avg_coherence_lengths['A1']],
    'A2': [sd_avg_coherence_lengths['A2'] - min(avg_coherence_lengths['A2d'], avg_coherence_lengths['A2s']), 
           max(avg_coherence_lengths['A2d'], avg_coherence_lengths['A2s']) - sd_avg_coherence_lengths['A2']],
    'A3': [sd_avg_coherence_lengths['A3'] - min(avg_coherence_lengths['A3d'], avg_coherence_lengths['A3s']), 
           max(avg_coherence_lengths['A3d'], avg_coherence_lengths['A3s']) - sd_avg_coherence_lengths['A3']]
}

thickness_error_fractions = (thickness_error / np.array(list(thicknesses.values())))
sd_error_fractions = {
    'A1': thickness_error_fractions[0],
    'A2': thickness_error_fractions[2],
    'A3': thickness_error_fractions[4]
}

sd_summed_peak_area_errors = {
    'A1': sd_summed_peak_areas['A1'] * sd_error_fractions['A1'],
    'A2': sd_summed_peak_areas['A2'] * sd_error_fractions['A2'],
    'A3': sd_summed_peak_areas['A3'] * sd_error_fractions['A3']
}

display(sd_avg_dspacings)
display(sd_avg_coherence_lengths)
display(sd_summed_peak_areas)
# sd_dspacing_errors
# sd_coherence_length_errors

In [ ]:
sd_summed_peak_areas

In [ ]:
sd_summed_peak_area_errors

In [ ]:
sd_rDoCs_err = {k:v/sd_summed_peak_areas['A3'] for k,v in sd_summed_peak_area_errors.items()}
sd_rDoCs_err

In [ ]:
sd_rDoCs = {k:v/sd_summed_peak_areas['A3'] for k,v in sd_summed_peak_areas.items()}
sd_rDoCs

In [ ]:
chi_max

In [ ]:
chi_min

In [ ]:
from matplotlib.patches import Patch

In [ ]:
# Plot average d-spacing, scherrer coherence length, summed peak area vs film
# Single triple bar plot 

films = list(sd_avg_dspacings.keys())
x = np.arange(len(films))  # the label locations
width = 0.25  # the width of the bars

fig, ax1 = plt.subplots(figsize=(6, 3), dpi=150)
# colors = ['#339649', '#964933', '#493396']
# colors = [(247/255, 150/255,  69/255), 
#           (191/255, 194/255, 194/255), 
#           ( 48/255, 134/255, 155/255)]
colors = plt.cm.viridis_r(np.linspace(0.05,1,3))

# Primary y-axis (avg_dspacings)
dspacing_err = np.array([[sd_dspacing_errors[k][0] for k in films], [sd_dspacing_errors[k][1] for k in films]])
bars1 = ax1.bar(x - width, sd_avg_dspacings.values(), width, label='D-spacing', 
                yerr=dspacing_err+0.004, capsize=4, alpha=0.7, color=colors, edgecolor='black', hatch='///')
ax1.set_xlabel('Acceptor film')
ax1.set_ylabel('D-spacing [Å]', color='black')
ax1.set_ybound(lower=3.55)
ax1.tick_params(axis='y', labelcolor='black')
ax1.set_xticks(x)
ax1.set_xticklabels(films)
ax1.set_title('AX film GIWAXS π-π fit results')

# Secondary y-axis (avg_coherence_lengths)
coherence_length_err = np.array([[sd_coherence_length_errors[k][0] for k in films], 
                                 [sd_coherence_length_errors[k][1] for k in films]])
ax2 = ax1.twinx()
bars2 = ax2.bar(x, sd_avg_coherence_lengths.values(), width, label='CCL', color=colors, alpha=0.7, 
                yerr=coherence_length_err, capsize=4, edgecolor='black', hatch='oo')
ax2.set_ylabel('CCL [Å]', color='black')
ax2.set_ybound(lower=14.5)
ax2.tick_params(axis='y', labelcolor='black')

# # Third y-axis (summed_peak_areas)
# ax3 = ax1.twinx()
# ax3.spines['right'].set_position(('outward', 60))  # Offset the third axis
# bars3 = ax3.bar(x + width, sd_summed_peak_areas.values(), width, label='Peak area', color=colors[2], alpha=0.7, 
#                 yerr=sd_summed_peak_area_errors.values())
# ax3.set_ylabel('Intensity [arb. units]', color=colors[2])
# ax3.set_ybound(lower=0.08)
# ax3.tick_params(axis='y', labelcolor=colors[2])
# Third y-axis (rDoCs from summed_peak_areas)
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('outward', 60))  # Offset the third axis
bars3 = ax3.bar(x + width, sd_rDoCs.values(), width, label='rDoC', color=colors, alpha=0.7, 
                yerr=sd_rDoCs_err.values(), capsize=4, edgecolor='black', hatch='---')
ax3.set_ylabel(f'rDoC [no units]', color='black')
ax3.set_ybound(lower=0.08)
ax3.tick_params(axis='y', labelcolor='black')

# Adding legends
fig.tight_layout()  # Adjust subplots to fit in figure area.
# fig.legend(loc='upper center', bbox_to_anchor=(0.255, 0.89), fontsize=12)
# fig.legend(loc='upper left', fontsize=10)
# Create manual legend handles
legend_handles = [
    Patch(facecolor='lightgray', edgecolor='black', label='D-spacing', hatch='///'),
    Patch(facecolor='lightgray', edgecolor='black', label='CCL', hatch='oo'),
    Patch(facecolor='lightgray', edgecolor='black', label='rDoC', hatch='---')
]

# Adding legends with manual handles
fig.legend(handles=legend_handles, loc='upper center', bbox_to_anchor=(0.255, 0.89), fontsize=12)


# # Save
# parent_folder = 'pi-pi_fit_results_plots_v4'
# outPath.joinpath(parent_folder).mkdir(exist_ok=True)
# savePath = outPath.joinpath(parent_folder, f'chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
# savePath.mkdir(exist_ok=True)
# fig.savefig(savePath.joinpath(f'bar_plot_summary_pi-pi_fit.png'), dpi=150)

# Show plot
plt.show()
plt.close('all')

In [ ]:
"*+-./OX\ox|"

In [ ]:
sd_avg_dspacings

In [ ]:
sd_rDoCs

In [ ]:
# Plot average d-spacing, scherrer coherence length, summed peak area vs film
# Single triple bar plot 

films = list(avg_dspacings.keys())
x = np.arange(len(films))  # the label locations
width = 0.25  # the width of the bars

fig, ax1 = plt.subplots(figsize=(10, 4))

# Primary y-axis (avg_dspacings)
bars1 = ax1.bar(x - width, avg_dspacings.values(), width, label='D-spacing [Å]')
ax1.set_xlabel('Film')
ax1.set_ylabel('D-spacing [Å]', color='b')
ax1.set_ybound(lower=3.6)
ax1.tick_params(axis='y', labelcolor='b')
ax1.set_xticks(x)
ax1.set_xticklabels([sn[film] for film in films])
ax1.set_title('Weighted average d-spacings & coherence lengths; \n Total peak areas, normalized by thickness',
              x=0.28)

# Secondary y-axis (avg_coherence_lengths)
ax2 = ax1.twinx()
bars2 = ax2.bar(x, avg_coherence_lengths.values(), width, label='CCL [Å]', color='g', alpha=0.7)
ax2.set_ylabel('CCL [Å]', color='g')
ax2.set_ybound(lower=11.5)
ax2.tick_params(axis='y', labelcolor='g')

# Third y-axis (summed_peak_areas)
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('outward', 60))  # Offset the third axis
bars3 = ax3.bar(x + width, summed_peak_areas.values(), width, label='Intensity [arb. units]', color='r', alpha=0.7)
ax3.set_ylabel('Intensity [arb. units]', color='r')
ax3.set_ybound(lower=0.2, upper=0.4)
ax3.tick_params(axis='y', labelcolor='r')

# Adding legends
fig.tight_layout()  # Adjust subplots to fit in figure area.
fig.legend(loc='upper center', bbox_to_anchor=(0.75, 0.98), ncol=3)

# Show plot
plt.show()
plt.close('all')

# Display the data
display(avg_dspacings)
display(avg_coherence_lengths)
display(summed_peak_areas)


In [ ]:
# Plot average d-spacing, scherrer coherence length, summed peak area vs film
fig_dspacing, ax_dspacing = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
fig_ccl, ax_ccl = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
fig_area, ax_area = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)

ax_dspacing.scatter(list(avg_dspacings.keys()), list(avg_dspacings.values()))
ax_ccl.scatter(list(avg_coherence_lengths.keys()), list(avg_coherence_lengths.values()))
ax_area.scatter(list(summed_peak_areas.keys()), list(summed_peak_areas.values()))

ax_dspacing.set(title='Peak-area-weighted average d-spacings', xlabel='Film', ylabel='D-spacing [Å]')
ax_ccl.set(title='Peak-area-weighted average coherence lengths', xlabel='Film', ylabel='CCL [Å]')
ax_area.set(title='Summed peak areas', xlabel='Film', ylabel='Intensity [arb. units]')

plt.show()
plt.close('all')
display(avg_dspacings)
display(avg_coherence_lengths)
display(summed_peak_areas)

### Low-Q line fitting
Use lmfit to perform the linefits

#### Low-Q lmfit functions

In [ ]:
def lowq_lmfit(sliced_DA):
    """
    Function currently utilizes global variables, define in notebook!
    
    Inputs:
    - sliced_DA: 1D data for a selected chi bin, already meaned over chi
    """    
    point_y = float(sliced_DA.sel(qr=slice(0.72,0.76)).mean('qr'))
    point_y = point_y - point_y*(0.5)

    x = sliced_DA.qr.data
    y = sliced_DA.data

    # Define all models to include in fitting
    bkg_mod = models.LinearModel(prefix='bkg_')
    pars = bkg_mod.make_params(intercept=point_y, slope=0)
    pars['bkg_intercept'].set(vary=False)
    pars['bkg_slope'].set(vary=False)

    lowq_mod = models.PseudoVoigtModel(prefix='lowq_')  # lowq peak
    pars += lowq_mod.guess(y, x, center=0.43)
    pars['lowq_amplitude'].set(min=0)
    pars['lowq_center'].set(min=0.35, max=0.55)

    # Combine into full model
    mod = bkg_mod + lowq_mod 

    # Run fit and store all info in a ModelResult object
    out = mod.fit(y, pars, x=x)
    return out

def lowq_fit(sliced_DA):   
    # Run lmfit
    sliced_DA = sliced_DA.where(~(sliced_DA!=sliced_DA))
    out = lowq_lmfit(sliced_DA)
    FWHM = np.round(float(out.params['lowq_fwhm']), 2)
    Lc = np.round((2*np.pi*0.9)/float(out.params['lowq_fwhm']), 2)
    center = np.round(float(out.params['lowq_center']), 2)
    dspacing = np.round((2*np.pi)/float(out.params['lowq_center']), 2)
    
    amplitude = np.round(float(out.params['lowq_amplitude']), 3)

    chi_range = f'{str(round(sliced_DA.chi_bin.left))}-{str(round(sliced_DA.chi_bin.right))}'
    
    # rsquared = np.round(out.rsquared, 3)
    redchi = np.round(out.redchi, 5)

    AA1 = '$\\AA^{-1}$'
    
    # Plot
    q = sliced_DA.qr.data
    I = sliced_DA.data
    
    fig, ax = plt.subplots()
    fig.set(size_inches=(6.5,3.5), dpi=120, tight_layout=True)
       
    ax.plot(q, I, label='data', linewidth=2.5)
    ax.plot(q, out.best_fit, '--', label='full_fit')
    for key in out.eval_components():
        ax.plot(q, out.eval_components()[key], label=f'{key}')
    ax.set(xlabel=f'Q [{AA1}]', ylabel='Intensity [arb. units]', yscale='linear')
    ax.set_title(
        f'Low Q peak fit: {sn[sliced_DA.film]}, {chi_range}° chi; peak area = {amplitude}\n' + 
        f'center = {center}, FWHM = {FWHM} {AA1} (d-spacing = {dspacing}, $L_c$ = {Lc} $\\AA$)', 
        x=0.6)
    ax.legend(title='$\\chi_{red}$ '+f'= {redchi}', loc='upper left', bbox_to_anchor=(1,1))
    ax.grid(visible=True, axis='x')
    ax.xaxis.set_minor_locator(plt.MultipleLocator(0.1))
    
    return out, fig, ax

#### Run Low-Q fits

In [ ]:
# Peak fitting for each chi bin
DS = corr_DS.copy()

# Set chi bounds & bins, q bounds, etc.
# chi_min, chi_width, chi_bins = [14, 9.25, 8]  # for full chi cut
chi_min, chi_width, chi_bins = [14, 4, 18]  # for full chi cut, precise wedges
# chi_min, chi_width, chi_bins = [14, 4, 6]  # for π-π chi cut -> 38
# chi_min, chi_width, chi_bins = [14, 4, 7]  # for π-π chi cut -> 42
chi_max = chi_min + (chi_width * chi_bins)
colors = plt.cm.viridis_r(np.linspace(0.15,1,chi_bins))

# # q_min = 0.1  # for full qr cut
# q_min = 0.9  # for π-π qr cut
# q_max = 2.07

# For low-q peak
q_min = 0.17
q_max = 0.8

# Select attributes
# selected_attrs_dict = {'film': ['A2s', 'A2d', 'A3s', 'A3d']}
selected_attrs_dict = {}
selected_DAs = select_attrs(DS.data_vars.values(), selected_attrs_dict)

# For each selected DA, fit the π-π peak
all_outs = {}
all_params = {}
for DA in tqdm(selected_DAs):
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min,q_max))
    binned_DA = sliced_DA.groupby_bins('chi', chi_bins).sum('chi')
    outs = {}
    params = {}
    for i, chi_bin in enumerate(binned_DA.chi_bins.data[::-1]):
        key = f'{str(round(chi_bin.left))}-{str(round(chi_bin.right))}'
        sel_DA = binned_DA.sel(chi_bins=chi_bin)
        sel_DA.attrs['chi_bin'] = chi_bin
        outs[key], fig, ax = lowq_fit(sel_DA)
        params[key] = outs[key].params.valuesdict()

        # # Save
        # outPath.joinpath('lowq_linefits_v1').mkdir(exist_ok=True)
        # savePath = outPath.joinpath('lowq_linefits_v1', f'chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
        # savePath.mkdir(exist_ok=True)
        # fig.savefig(savePath.joinpath(f'{DA.film}_chi{key}_lowq_fit.png'), dpi=150)

        # plt.show()
        plt.close('all')
        
    all_outs[DA.film] = outs
    all_params[DA.film] = params

# # Save model_params.json in same save folder
# json_data = json.dumps(all_params)
# with open(str(savePath.joinpath('model_params.json')), 'w') as f:
#     f.write(json_data)

# # Create full fit reports folder and save entire report .txt files inside:
# savePath.joinpath('fit_reports').mkdir(exist_ok=True)
# for film, outs in all_outs.items():
#     for chi_bin, out in outs.items():
#         with savePath.joinpath('fit_reports', f'{film}_{chi_bin}_fit_result.txt').open(mode='w') as f:
#             f.write(out.fit_report())

#### Plot fit results

In [ ]:
# # Load params from json file
# # chi_width = 4
# # chi_bins = 6
# # chi_range = '14-38'
# # chi_width = 4
# # chi_bins = 7
# # chi_range = '14-42'
# # chi_width = 4
# # chi_bins = 18
# # chi_range = '14-86'

# with open(str(outPath.joinpath('pi-pi_linefits_v3', f'chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_range}', 'model_params.json')), 'r') as f:
#     all_params = json.load(f)

# len(all_params)

In [ ]:
# Plot pseudovoigt fractions, d-spacings, scherrer coherence lengths, and normalized peak areas vs chi

# Initialize figures to be populated later
fig_psvoigt, ax_psvoigt = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
fig_dspacing, ax_dspacing = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
fig_ccl, ax_ccl = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
fig_pole, ax_pole = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)

# Define custom colors for each film, here I've chosen just two shades along the same sequention colorbar for each sample group
colors_dict = {
    'A1s':plt.cm.Blues(np.linspace(0.6,0.8,2))[0],
    'A1d':plt.cm.Blues(np.linspace(0.6,0.8,2))[1],
    'A2s':plt.cm.Greens(np.linspace(0.6,0.8,2))[0],
    'A2d':plt.cm.Greens(np.linspace(0.6,0.8,2))[1],
    'A3s':plt.cm.Oranges(np.linspace(0.6,0.8,2))[0],
    'A3d':plt.cm.Oranges(np.linspace(0.6,0.8,2))[1]
}

### Extracting info from all_params & populating ___ vs chi plots:
summed_peak_areas = {}
avg_dspacings = {}
avg_coherence_lengths = {}
for film in all_params:
    peak_areas = []
    pvoigt_fracs = []
    peak_centers = []
    peak_fwhms = []
    chis = []
    for chi_bin, params in all_params[film].items():
        # Get starting value of chi bin
        chi = int(chi_bin.split('-')[0])
        chis.append(chi)
        # Get peak areas
        peak_area = params['lowq_amplitude']
        peak_areas.append(peak_area)
        # Get pseudovoigt fraction
        pvoigt_frac = params['lowq_fraction']
        pvoigt_fracs.append(pvoigt_frac)
        # Get peak centers
        peak_center = params['lowq_center']
        peak_centers.append(peak_center)
        # Get peak fwhms
        peak_fwhm = params['lowq_fwhm']
        peak_fwhms.append(peak_fwhm)
        
    # Get normalized peak areas (divide by sum of all peak areas; adds to 1) and write area sum to dict
    peak_areas = np.array(peak_areas)
    summed_peak_areas[film] = np.sum(peak_areas)  # Write sum of peak area out, this corresponds to relative extent of crystallinity if thickness normalized
    normed_peak_areas = peak_areas / np.sum(peak_areas)

    # Calculate d-spacings from peak centers, calculate peak-area-weighted average of d-spacing
    dspacings = (2*np.pi) / np.array(peak_centers) 
    avg_dspacing = np.round(np.sum(dspacings*normed_peak_areas), 2)
    avg_dspacings[film] = avg_dspacing  

    # Calculate coherence length from peak fwhm, calculate peak-area-weighted average of coherence length    
    coherence_lengths = (2*np.pi*0.9) / np.array(peak_fwhms) 
    avg_coherence_length = np.round(np.sum(coherence_lengths*normed_peak_areas), 2)
    avg_coherence_lengths[film] = avg_coherence_length

    ### Plotting
    # Plot pseudovoigt fraction vs chi for each film
    ax_psvoigt.plot(chis, pvoigt_fracs, label=film, color=colors_dict[film], marker='o')
    ax_psvoigt.set(title='Pseudo-voigt lorentzian-to-gaussian ratio versus $\\chi$',
                   ylabel='Lorentzian peak fraction',
                   xlabel='Binned $\\chi$ value [°]')
    ax_psvoigt.legend(title='Film', loc='upper left', bbox_to_anchor=(1,1))

    # Plot d-spacing vs chi for each film
    ax_dspacing.plot(chis, dspacings, label=film, color=colors_dict[film], marker='o')
    ax_dspacing.set(title='D-spacing versus $\\chi$',
                    ylabel='d-spacing [Å]',
                    xlabel='Binned $\\chi$ value [°]')
    ax_dspacing.legend(title='Film', loc='upper left', bbox_to_anchor=(1,1))

    # Plot coherence length vs chi for each film
    ax_ccl.plot(chis, coherence_lengths, label=film, color=colors_dict[film], marker='o')
    ax_ccl.set(title='Crystalline coherence length (CCL) versus $\\chi$',
               ylabel='CCL [Å]',
               xlabel='Binned $\\chi$ value [°]')
    ax_ccl.legend(title='Film', loc='upper left', bbox_to_anchor=(1,1))
    
    # Plot 'pole figure' for each film
    ax_pole.plot(chis, normed_peak_areas, label=film, color=colors_dict[film], marker='o')
    ax_pole.set(title='Normalized π-π peak areas versus $\\chi$:\n'+
                 'peak area divided by sum of all peak areas ',
                ylabel='Peak area [arb. units]',
                xlabel='Binned $\\chi$ value [°]')
    ax_pole.legend(title='Film', loc='upper left', bbox_to_anchor=(1,1.1))    

    # Extract chi values for file saving purposes
    chi_width = chis[0]-chis[1]
    chi_bins = len(chis)
    chi_min = chis[-1]
    chi_max = chis[0]+chi_width

    # # Save
    # parent_folder = 'lowq_fit_results_plots_v1'
    # outPath.joinpath(parent_folder).mkdir(exist_ok=True)
    # savePath = outPath.joinpath(parent_folder, f'chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
    # savePath.mkdir(exist_ok=True)
    # fig_psvoigt.savefig(savePath.joinpath(f'pseudovoigt_ratios_lowq_fit.png'), dpi=150)
    # fig_dspacing.savefig(savePath.joinpath(f'dspacings_lowq_fit.png'), dpi=150)
    # fig_ccl.savefig(savePath.joinpath(f'coherence_lengths_lowq_fit.png'), dpi=150)
    # fig_pole.savefig(savePath.joinpath(f'peak_areas_lowq_fit.png'), dpi=150)
    
plt.show()
plt.close('all')  

In [ ]:
# Average dspacing for each of two duplicate measurements
sd_avg_dspacings = {
    'A1': (avg_dspacings['A1d'] + avg_dspacings['A1s'])/2,
    'A2': (avg_dspacings['A2d'] + avg_dspacings['A2s'])/2,
    'A3': (avg_dspacings['A3d'] + avg_dspacings['A3s'])/2 
}

sd_avg_coherence_lengths = {
    'A1': (avg_coherence_lengths['A1d'] + avg_coherence_lengths['A1s'])/2,
    'A2': (avg_coherence_lengths['A2d'] + avg_coherence_lengths['A2s'])/2,
    'A3': (avg_coherence_lengths['A3d'] + avg_coherence_lengths['A3s'])/2 
}

sd_summed_peak_areas = {
    'A1': (summed_peak_areas['A1d'] + summed_peak_areas['A1s'])/2,
    'A2': (summed_peak_areas['A2d'] + summed_peak_areas['A2s'])/2,
    'A3': (summed_peak_areas['A3d'] + summed_peak_areas['A3s'])/2 
}

sd_dspacing_errors = {
    'A1': [sd_avg_dspacings['A1'] - min(avg_dspacings['A1d'], avg_dspacings['A1s']), 
           max(avg_dspacings['A1d'], avg_dspacings['A1s']) - sd_avg_dspacings['A1']],
    'A2': [sd_avg_dspacings['A2'] - min(avg_dspacings['A2d'], avg_dspacings['A2s']), 
           max(avg_dspacings['A2d'], avg_dspacings['A2s']) - sd_avg_dspacings['A2']],
    'A3': [sd_avg_dspacings['A3'] - min(avg_dspacings['A3d'], avg_dspacings['A3s']), 
           max(avg_dspacings['A3d'], avg_dspacings['A3s']) - sd_avg_dspacings['A3']]
}

sd_coherence_length_errors = {
    'A1': [sd_avg_coherence_lengths['A1'] - min(avg_coherence_lengths['A1d'], avg_coherence_lengths['A1s']), 
           max(avg_coherence_lengths['A1d'], avg_coherence_lengths['A1s']) - sd_avg_coherence_lengths['A1']],
    'A2': [sd_avg_coherence_lengths['A2'] - min(avg_coherence_lengths['A2d'], avg_coherence_lengths['A2s']), 
           max(avg_coherence_lengths['A2d'], avg_coherence_lengths['A2s']) - sd_avg_coherence_lengths['A2']],
    'A3': [sd_avg_coherence_lengths['A3'] - min(avg_coherence_lengths['A3d'], avg_coherence_lengths['A3s']), 
           max(avg_coherence_lengths['A3d'], avg_coherence_lengths['A3s']) - sd_avg_coherence_lengths['A3']]
}

thickness_error_fractions = (thickness_error / np.array(list(thicknesses.values())))
sd_error_fractions = {
    'A1': error_fractions[0],
    'A2': error_fractions[2],
    'A3': error_fractions[4]
}

sd_summed_peak_area_errors = {
    'A1': sd_summed_peak_areas['A1'] * sd_error_fractions['A1'],
    'A2': sd_summed_peak_areas['A2'] * sd_error_fractions['A2'],
    'A3': sd_summed_peak_areas['A3'] * sd_error_fractions['A3']
}

# sd_avg_dspacings
# sd_dspacing_errors
sd_coherence_length_errors

In [ ]:
# Plot average d-spacing, scherrer coherence length, summed peak area vs film
# Single triple bar plot 

films = list(sd_avg_dspacings.keys())
x = np.arange(len(films))  # the label locations
width = 0.25  # the width of the bars

fig, ax1 = plt.subplots(figsize=(6, 3), dpi=150)
colors = ['#339649', '#964933', '#493396']

# Primary y-axis (avg_dspacings)
dspacing_err = np.array([[sd_dspacing_errors[k][0] for k in films], [sd_dspacing_errors[k][1] for k in films]])
bars1 = ax1.bar(x - width, sd_avg_dspacings.values(), width, label='D-spacing [Å]', 
                yerr=dspacing_err, color=colors[0])
ax1.set_xlabel('Acceptor (averaged static & dyanmic film)')
ax1.set_ylabel('D-spacing [Å]', color=colors[0])
ax1.set_ybound(lower=12)
ax1.tick_params(axis='y', labelcolor=colors[0])
ax1.set_xticks(x)
ax1.set_xticklabels(films)
ax1.set_title('Weighted average d-spacings & coherence lengths; \n total peak areas, normalized by thickness')

# Secondary y-axis (avg_coherence_lengths)
coherence_length_err = np.array([[sd_coherence_length_errors[k][0] for k in films], 
                                 [sd_coherence_length_errors[k][1] for k in films]])
ax2 = ax1.twinx()
bars2 = ax2.bar(x, sd_avg_coherence_lengths.values(), width, label='CCL [Å]', color=colors[1], alpha=0.7, yerr=coherence_length_err)
ax2.set_ylabel('CCL [Å]', color=colors[1])
ax2.set_ybound(lower=20)
ax2.tick_params(axis='y', labelcolor=colors[1])

# Third y-axis (summed_peak_areas)
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('outward', 60))  # Offset the third axis
bars3 = ax3.bar(x + width, sd_summed_peak_areas.values(), width, label='Peak area', color=colors[2], alpha=0.7, 
                yerr=sd_summed_peak_area_errors.values())
ax3.set_ylabel('Intensity [arb. units]', color=colors[2])
ax3.set_ybound(lower=0.7)
ax3.tick_params(axis='y', labelcolor=colors[2])

# Adding legends
fig.tight_layout()  # Adjust subplots to fit in figure area.
fig.legend(loc='upper center', bbox_to_anchor=(0.255, 0.835), fontsize=10)


# Save
parent_folder = 'lowq_fit_results_plots_v1'
outPath.joinpath(parent_folder).mkdir(exist_ok=True)
savePath = outPath.joinpath(parent_folder, f'chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
savePath.mkdir(exist_ok=True)
fig.savefig(savePath.joinpath(f'bar_plot_summary_lowq_fit.png'), dpi=150)

# Show plot
plt.show()
plt.close('all')

### Pole Figures

#### Pi-pi 

In [ ]:
### Set limits
q_min = 1.68
q_max = 1.95
chi_min = 12
chi_max = 80  # to ~45 least until I mask out the silicon(?) background scattering that interferes


# selected_attrs_dict = {'material':['PM6'], 'incident_angle':['th0.120']}
# # selected_attrs_dict = {}
# selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)
good_scans_list = ['sam1_10s', 'sam2_10s-2', 'sam3_30s', 'sam4_30s', 'sam5_30s', 'sam6_2s']
selected_DAs = []
for DA in DS.data_vars.values():
    sample_name = DA.attrs['sample_id']
    exp_time = DA.attrs['exposure_time']
    if f'{sample_name}_{exp_time}' in good_scans_list:
        selected_DAs.append(DA)
        
peak_areas_dict = {}    
for DA in tqdm(selected_DAs):
    chis = DA.chi.sel(chi=slice(chi_min,chi_max)).data
    
    peak_areas = np.array([])
    peak_centers = np.array([])
    
    for chi in chis:
        sliced_DA = DA.sel(qr=slice(q_min-0.1, q_max+0.1), chi=float(chi))

        points_x = [q_min, q_max]
        points_y = [float(sliced_DA.sel(qr=slice(points_x[0]-0.05, points_x[0]+0.05)).mean('qr')), 
                    float(sliced_DA.sel(qr=slice(points_x[1]-0.05, points_x[1]+0.05)).mean('qr'))]
        m = (points_y[1]-points_y[0])/(points_x[1]-points_x[0])
        b = points_y[1] - (m*points_x[1])
        y_fit = np.polyval([m, b], sliced_DA.qr)
        
        sub_sliced_DA = sliced_DA-y_fit

        peak_area = sub_sliced_DA.sel(qr=slice(q_min,q_max)).integrate('qr')
        peak_area = np.nan if peak_area<0 else peak_area  # remove negatives as they are nonphysical
        
        peak_areas = np.append(peak_areas, float(peak_area))
                        
#         # Plot linecuts for each chi
#         fig, ax = plt.subplots()
#         sliced_DA.plot(ax=ax)
#         ax.plot(points_x, points_y)
#         plt.show()

#         plt.close('all')

    normed_peak_areas = peak_areas / np.nansum(peak_areas)
    data = np.vstack((chis, normed_peak_areas)).T
    peak_areas_dict[DA.attrs['sample_id']+'_'+DA.attrs['exposure_time']] = data

    # Individual measurement pole figure plot
    fig, ax = plt.subplots(figsize=(5,3.3))
    fig.set(dpi=120)
    
    ax.plot(chis, normed_peak_areas)
    fig.suptitle(f'{DA.sample_id} {DA.exposure_time} Pole Figure; $\\alpha_i = $ {DA.incident_angle[2:-1]}°', x=0.54)
    ax.set(title=f'{q_min} to {q_max} 1/Å Q Bounds; {chi_min}° to {chi_max}° $\chi$')
    ax.set(xlabel='$\chi$ [°]', ylabel='Integrated Pi-Pi Peak Area [arb. units]')
    plt.subplots_adjust(top=0.85, bottom=0.2, left=0.2)
    
    # fig.savefig(savePath.joinpath(f'{DA.material}-{DA.solvent}-{DA.rpm}_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{DA.incident_angle}.png'), dpi=150)
    # fig.savefig(savePath.joinpath(f'{DA.material}-{DA.solvent}_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{DA.incident_angle}.png'), dpi=150)

    plt.show()
    
    plt.close('all')

In [ ]:
### Set savePath
# savePath = outPath.joinpath('var/pipi_pole_figures_v1')
savePath = outPath.joinpath('fix/pipi_pole_figures_v1')

# for th in tqdm(['th0.080', 'th0.100', 'th0.120', 'th0.140']):
for th in tqdm(['th0.120']):
    selected_dict_items = [(name, data) for (name, data) in peak_areas_dict.items()]
    # selected_dict_items = [(name, data) for (name, data) in peak_areas_dict.items() if 
    #                        # # 'Y7BO-' in name and
    #                        # 'CF' in name and '3000' in name and 
    #                        th in name]    
    #                        # 'CFCN-' in name]    


    fig, ax = plt.subplots(figsize=(5,3.3))
    fig.set(dpi=120)
    
    colors = plt.cm.Dark2(np.linspace(0,1,len(selected_dict_items)))

    for i, (measurement, pole_fig_arr_data) in enumerate(selected_dict_items):
        chis = pole_fig_arr_data[:,0]
        normed_peak_areas = pole_fig_arr_data[:,1]
        ax.plot(chis, normed_peak_areas, color=colors[i], label=measurement)
        
    fig.suptitle('Pole Figure', x=0.55)
    ax.set(title=f'{q_min} to {q_max} 1/Å Q Fit Region; {chi_min}° to {chi_max}° $\chi$')
    ax.set(xlabel='$\chi$ [°]', ylabel='Normalized Peak Area')
    ax.set_xlim(None,40)
    ax.set_ylim(None,0.14)

    
    plt.subplots_adjust(top=0.85, bottom=0.2, left=0.2)
    ax.legend()
    
    savePath = outPath.joinpath('pipi_pole_figures_v1')
    savePath.mkdir(exist_ok=True)
    fig.savefig(savePath.joinpath(f'all_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{th}.png'), dpi=150)
    
    plt.show()
    plt.close('all')

#### Lamella

In [ ]:
### Set limits
q_min = 0.2
q_max = 0.45
chi_min = 10
chi_max = 82  # to ~45 least until I mask out the silicon(?) background scattering that interferes

# Set savePath
savePath = outPath.joinpath('PM6-Y6series/lamella_pole_figures_v1')

selected_attrs_dict = {'material':['PM6'], 'incident_angle': ['th0.120']}
selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)
 
peak_areas_dict = {}    
for DA in tqdm(selected_DAs):
    chis = DA.chi.sel(chi=slice(chi_min,chi_max)).data
    
    peak_areas = np.array([])
    peak_centers = np.array([])
    
    for chi in chis:
        sliced_DA = DA.sel(qr=slice(q_min-0.05, q_max+0.05), chi=float(chi))

        points_x = [q_min, q_max]
        points_y = [float(sliced_DA.sel(qr=slice(points_x[0]-0.05, points_x[0]+0.05)).mean('qr')), 
                    float(sliced_DA.sel(qr=slice(points_x[1]-0.05, points_x[1]+0.05)).mean('qr'))]
        # points_y = [float(sliced_DA.sel(qr=q_min, method='nearest')), 
        #             float(sliced_DA.sel(qr=q_max, method='nearest'))]
        m = (points_y[1]-points_y[0])/(points_x[1]-points_x[0])
        b = points_y[1] - (m*points_x[1])
        y_fit = np.polyval([m, b], sliced_DA.qr)
        
        sub_sliced_DA = sliced_DA-y_fit

        peak_area = sub_sliced_DA.sel(qr=slice(q_min,q_max)).integrate('qr')

        # peak_area = sliced_DA.sel(qr=slice(None,0.45)).integrate('qr')
        peak_area = np.nan if peak_area<0 else peak_area  # remove negatives as they are nonphysical
        
        peak_areas = np.append(peak_areas, float(peak_area))
                        
#         # Plot linecuts for each chi
#         fig, ax = plt.subplots()
#         sliced_DA.plot(ax=ax)
#         ax.plot(points_x, points_y)
#         plt.show()

#         plt.close('all')

    normed_peak_areas = peak_areas / np.nansum(peak_areas)
    data = np.vstack((chis, normed_peak_areas)).T
    # peak_areas_dict[DA.attrs['material']+'-'+DA.attrs['solvent']+'-'+DA.attrs['rpm']+'-'+DA.attrs['incident_angle']] = data
    peak_areas_dict[DA.attrs['material']+'-'+DA.attrs['solvent']+'-'+DA.attrs['incident_angle']] = data


    # Individual measurement pole figure plot
    fig, ax = plt.subplots(figsize=(5,3.3))
    fig.set(dpi=120)
    
    ax.plot(chis, normed_peak_areas)
    # fig.suptitle(f'{DA.material} {DA.solvent} {DA.rpm} Pole Figure; $\\alpha_i = $ {DA.incident_angle[2:-1]}°', x=0.54)
    fig.suptitle(f'{DA.material} {DA.solvent} Pole Figure; $\\alpha_i = $ {DA.incident_angle[2:-1]}°', x=0.54)

    ax.set(title=f'{q_min} to {q_max} 1/Å Q Bounds; {chi_min}° to {chi_max}° $\chi$')
    ax.set(xlabel='$\chi$ [°]', ylabel='Integrated Pi-Pi Peak Area [arb. units]')
    plt.subplots_adjust(top=0.85, bottom=0.2, left=0.2)
    
    # fig.savefig(savePath.joinpath(f'{DA.material}-{DA.solvent}-{DA.rpm}_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{DA.incident_angle}.png'), dpi=150)
    # fig.savefig(savePath.joinpath(f'{DA.material}-{DA.solvent}_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{DA.incident_angle}.png'), dpi=150)
    
    plt.show()
    
    plt.close('all')

In [ ]:
### Set savePath
# savePath = outPath.joinpath('PM6-Y6series/var/lamella_pole_figures_v1')
savePath = outPath.joinpath('fix/lamella_pole_figures_v1')

# for th in tqdm(['th0.080', 'th0.100', 'th0.120', 'th0.140']):
for th in tqdm(['th0.120']):
    selected_dict_items = [(name, data) for (name, data) in peak_areas_dict.items() if 
                           # # 'Y7BO-' in name and
                           # 'CF' in name and '3000' in name and 
                           # 'CF-' in name]     
                           th in name] 

    fig, ax = plt.subplots(figsize=(5,3.3))
    fig.set(dpi=120)
    
    colors = plt.cm.Dark2(np.linspace(0,1,len(selected_dict_items)))

    for i, (measurement, pole_fig_arr_data) in enumerate(selected_dict_items):
        chis = pole_fig_arr_data[:,0]
        normed_peak_areas = pole_fig_arr_data[:,1]
        ax.plot(chis, normed_peak_areas, color=colors[i], label=measurement)
        
    fig.suptitle('Pole Figure', x=0.55)
    ax.set(title=f'{q_min} to {q_max} 1/Å Q Fit Region ; {chi_min}° to {chi_max}° $\chi$')
    ax.set(xlabel='$\chi$ [°]', ylabel='Normalized Peak Area')
    
    plt.subplots_adjust(top=0.85, bottom=0.2, left=0.2)
    ax.legend()

    # fig.savefig(savePath.joinpath(f'multiple-PM6_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{th}.png'), dpi=150)


    plt.show()
    plt.close('all')

#### Backbone

In [ ]:
### Set limits
q_min = 0.55
q_max = 0.77
chi_min = 10
chi_max = 82  # to ~45 least until I mask out the silicon(?) background scattering that interferes

# Set savePath
savePath = outPath.joinpath('PM6-Y6series/lamella_pole_figures_v1')

selected_attrs_dict = {'material':['PM6'], 'incident_angle': ['th0.120']}
selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)
 
peak_areas_dict = {}    
for DA in tqdm(selected_DAs):
    chis = DA.chi.sel(chi=slice(chi_min,chi_max)).data
    
    peak_areas = np.array([])
    peak_centers = np.array([])
    
    for chi in chis:
        sliced_DA = DA.sel(qr=slice(q_min, q_max), chi=float(chi))

        points_x = [q_min, q_max]
        points_y = [float(sliced_DA.sel(qr=slice(points_x[0]-0.05, points_x[0]+0.05)).mean('qr')), 
                    float(sliced_DA.sel(qr=slice(points_x[1]-0.05, points_x[1]+0.05)).mean('qr'))]
        # points_y = [float(sliced_DA.sel(qr=q_min, method='nearest')), 
        #             float(sliced_DA.sel(qr=q_max, method='nearest'))]
        m = (points_y[1]-points_y[0])/(points_x[1]-points_x[0])
        b = points_y[1] - (m*points_x[1])
        y_fit = np.polyval([m, b], sliced_DA.qr)
        
        sub_sliced_DA = sliced_DA-y_fit

        peak_area = sub_sliced_DA.sel(qr=slice(q_min,q_max)).integrate('qr')

        # peak_area = sliced_DA.sel(qr=slice(None,0.45)).integrate('qr')
        peak_area = np.nan if peak_area<0 else peak_area  # remove negatives as they are nonphysical
        
        peak_areas = np.append(peak_areas, float(peak_area))
                        
#         # Plot linecuts for each chi
#         fig, ax = plt.subplots()
#         sliced_DA.plot(ax=ax)
#         ax.plot(points_x, points_y)
#         plt.show()

#         plt.close('all')

    normed_peak_areas = peak_areas / np.nansum(peak_areas)
    data = np.vstack((chis, normed_peak_areas)).T
    # peak_areas_dict[DA.attrs['material']+'-'+DA.attrs['solvent']+'-'+DA.attrs['rpm']+'-'+DA.attrs['incident_angle']] = data
    peak_areas_dict[DA.attrs['material']+'-'+DA.attrs['solvent']+'-'+DA.attrs['incident_angle']] = data


    # Individual measurement pole figure plot
    fig, ax = plt.subplots(figsize=(5,3.3))
    fig.set(dpi=120)
    
    ax.plot(chis, normed_peak_areas)
    # fig.suptitle(f'{DA.material} {DA.solvent} {DA.rpm} Pole Figure; $\\alpha_i = $ {DA.incident_angle[2:-1]}°', x=0.54)
    fig.suptitle(f'{DA.material} {DA.solvent} Pole Figure; $\\alpha_i = $ {DA.incident_angle[2:-1]}°', x=0.54)

    ax.set(title=f'{q_min} to {q_max} 1/Å Q Bounds; {chi_min}° to {chi_max}° $\chi$')
    ax.set(xlabel='$\chi$ [°]', ylabel='Integrated Pi-Pi Peak Area [arb. units]')
    plt.subplots_adjust(top=0.85, bottom=0.2, left=0.2)
    
    # fig.savefig(savePath.joinpath(f'{DA.material}-{DA.solvent}-{DA.rpm}_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{DA.incident_angle}.png'), dpi=150)
    # fig.savefig(savePath.joinpath(f'{DA.material}-{DA.solvent}_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{DA.incident_angle}.png'), dpi=150)
    
    plt.show()
    
    plt.close('all')

In [ ]:
# Set parameters:
q_min = 0.55
q_max = 0.77
chi_min = 45
chi_max = 82

# Set savePath
# savePath = outPath.joinpath('PM6-Y6series/var/backbone_pole_figures_v1')
savePath = outPath.joinpath('fix/backbone_pole_figures_v1')

# for th in tqdm(['th0.080', 'th0.100', 'th0.120', 'th0.140']):
for th in tqdm(['th0.120']):
    selected_dict_items = [(name, data) for (name, data) in peak_areas_dict.items() if 
                           # # 'Y7BO-' in name and
                           # 'CF' in name and '3000' in name and 
                           # 'CF-' in name]     
                           th in name] 

    fig, ax = plt.subplots(figsize=(5,3.3))
    fig.set(dpi=120)
    
    colors = plt.cm.Dark2(np.linspace(0,1,len(selected_dict_items)))

    for i, (measurement, pole_fig_arr_data) in enumerate(selected_dict_items):
        chis = pole_fig_arr_data[:,0]
        normed_peak_areas = pole_fig_arr_data[:,1]
        ax.plot(chis, normed_peak_areas, color=colors[i], label=measurement)
        
    fig.suptitle('Pole Figure', x=0.55)
    ax.set(title=f'{q_min} to {q_max} 1/Å Q Fit Region ; {chi_min}° to {chi_max}° $\chi$')
    ax.set(xlabel='$\chi$ [°]', ylabel='Normalized Peak Area')
    ax.set_xlim(45,None)
    
    plt.subplots_adjust(top=0.85, bottom=0.2, left=0.2)
    ax.legend()
    
    
    fig.savefig(savePath.joinpath(f'multiple-PM6_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{th}.png'), dpi=150)


    # plt.show()
    plt.close('all')

# Original notebook code

### 2D plots

#### Caked Images

In [ ]:
filename = 'fix_caked_stitched.zarr'
DS = xr.open_zarr(zarrsPath.joinpath(filename))
DS = DS.where(DS>1e-5)
DS

In [ ]:
# # How one could apply a sin chi correction
# sin_chi_DA = np.sin(np.radians(np.abs(DA.chi)))
# # sin_chi_DA

# corr_DA = DA * sin_chi_DA
# # corr_DA

In [ ]:
# A way to select dataarrays based on attribute values:
selected_DAs = [da for da in DS.data_vars.values() if 
                da.attrs['material']]
                # da.attrs['material'] in ['Y6', 'Y7'] and 
                # da.attrs['incident_angle'] in ['th0.120', 'th0.140']]

len(selected_DAs)

In [ ]:
# Plot and optionally save selected dataarrays:
# Set chi range: Full range
chi_min = -90
chi_max = 90

for DA in tqdm(selected_DAs):
    # Slice dataarray to select plotting region 
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max))
    
    # real_min = float(DA.sel(q_xy=slice(-0.5, -0.1), q_z=slice(0.1, 0.4)).compute().quantile(1e-3))
    real_min = float(DA.compute().quantile(0.01))
    cmin = 1 if real_min < 1 else real_min
    
    # cmax = float(DA.sel(q_xy=slice(-0.5, -0.1), q_z=slice(0.1, 2)).compute().quantile(1))   
    cmax = float(DA.compute().quantile(0.99))   
    
    # Plot sliced dataarray
    ax = sliced_DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), figsize=(5,4), interpolation='antialiased')  # plot, optional parameter interpolation='antialiased' for image smoothing
    ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)  # set colorbar label & parameters 
    ax.axes.set(title=f'Polar Plot: {DA.material} {DA.solvent}, {float(DA.incident_angle[2:])}° Incidence',
                xlabel='q$_r$ [Å$^{-1}$]', ylabel='$\chi$ [°]')  # set title, axis labels, misc
    ax.figure.set(tight_layout=True, dpi=130)  # Adjust figure dpi & plotting style
    
    plt.show()  # Comment to mute plotting output
    
    # Uncomment below line and set savepath/savename for saving plots, I usually like to check 
    # ax.figure.savefig(outPath.joinpath('PM6-Y6set_waxs', f'polar-2D_{DA.sample_id}_{chi_min}to{chi_max}chi_{DA.incident_angle}.png'), dpi=150)
    plt.close('all')

In [ ]:
# Plot and optionally save selected dataarrays:
# Set chi range: Full range
chi_min = -90
chi_max = 90

for DA in tqdm(selected_DAs):
    # Slice dataarray to select plotting region 
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max))
    
    # real_min = float(DA.sel(q_xy=slice(-0.5, -0.1), q_z=slice(0.1, 0.4)).compute().quantile(1e-3))
    real_min = float(DA.compute().quantile(0.01))
    cmin = 1 if real_min < 1 else real_min
    
    # cmax = float(DA.sel(q_xy=slice(-0.5, -0.1), q_z=slice(0.1, 2)).compute().quantile(1))   
    cmax = float(DA.compute().quantile(0.999))   
    
    # Plot sliced dataarray
    ax = sliced_DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), figsize=(5,4), interpolation='antialiased')  # plot, optional parameter interpolation='antialiased' for image smoothing
    ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)  # set colorbar label & parameters 
    ax.axes.set(title=f'Polar Plot: {DA.material} {DA.solvent} {DA.rpm}, {float(DA.incident_angle[2:])}° Incidence',
                xlabel='q$_r$ [Å$^{-1}$]', ylabel='$\chi$ [°]')  # set title, axis labels, misc
    ax.figure.set(tight_layout=True, dpi=130)  # Adjust figure dpi & plotting style
    
    plt.show()  # Comment to mute plotting output
    
    # Uncomment below line and set savepath/savename for saving plots, I usually like to check 
    # ax.figure.savefig(outPath.joinpath('PM6-Y6set_waxs', f'polar-2D_{DA.sample_id}_{chi_min}to{chi_max}chi_{DA.incident_angle}.png'), dpi=150)
    plt.close('all')

In [ ]:
# Plot and optionally save selected dataarrays:
# Set chi range: In plane slice, choose a smooth section without detector gap/edge effects
chi_min = 72
chi_max = 82

for DA in tqdm(selected_DAs):
    # Slice dataarray to select plotting region 
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(0.23,2.05))
    cmin = float(sliced_DA.compute().quantile(1e-2))  # Set color minimum value, based on quantile 
    cmax = float(sliced_DA.compute().quantile(1-1e-6))  # Set color maximum value, based on quantile
    
    # Plot sliced dataarray
    ax = sliced_DA.plot.imshow(cmap=cmap, norm=LogNorm(cmin, cmax), figsize=(5,4))  # plot
    ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)  # set colorbar label & parameters 
    ax.axes.set(title=f'Polar Plot: {DA.sample_id}, {float(DA.incident_angle[2:])}° Incidence',
                xlabel='q$_r$ [Å$^{-1}$]', ylabel='$\chi$ [°]')  # set title, axis labels, misc
    ax.figure.set(tight_layout=True, dpi=130)  # Adjust figure dpi & plotting style
    
    plt.show()  # Comment to mute plotting output
    
    # Uncomment below line and set savepath/savename for saving plots, I usually like to check 
    # ax.figure.savefig(outPath.joinpath('PM6-Y6set_waxs', f'polar-2D_{DA.sample_id}_{chi_min}to{chi_max}chi_{DA.incident_angle}.png'), dpi=150)
    plt.close('all')

In [ ]:
# # A way to save data as csv files  
# for DA in DS.data_vars.values():
#     # qr columns, chi rows
#     DA.to_pandas().to_csv(outPath.joinpath('PM6-Y6_waxs', f'polar-2D_{DA.polymer}-{DA.weight_percent}_{DA.incident_angle}_{DA.scan_id}.csv'))

In [ ]:
filename = 'fix_raw_stitched.zarr'
DS = xr.open_zarr(zarrsPath.joinpath(filename))
DS = DS.where(DS>1e-6)
DS

In [ ]:
selected_DAs = [da for da in DS.data_vars.values() if 
                da.attrs['incident_angle']]
len(selected_DAs)

In [ ]:
# Plot & optionally save each selected polymer: fix data set
for DA in tqdm(selected_DAs):
    # Slice data for selected q ranges (will need to rename q_xy if dimensions are differently named)
    sliced_DA = DA.copy()
    sliced_DA = DA.sel(pix_y=slice(None, 680), pix_x=slice(150,None))

    # real_min = float(DA.sel(q_xy=slice(-0.5, -0.1), q_z=slice(0.1, 0.4)).compute().quantile(1e-3))
    real_min = float(DA.compute().quantile(0.01))
    cmin = 1 if real_min < 1 else real_min
    
    # cmax = float(DA.sel(q_xy=slice(-0.5, -0.1), q_z=slice(0.1, 2)).compute().quantile(1))   
    cmax = float(DA.compute().quantile(0.999))   
    
    # Same plotting procedure as above
    # ax = sliced_DA.plot.imshow(cmap=cmap, norm=LogNorm(cmin, cmax), interpolation='antialiased', figsize=(5.5,3.3))
    ax = sliced_DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), interpolation='antialiased', figsize=(5.5,3.3),
                               origin='upper')

    ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)
    ax.axes.set(title=f'Raw Detector: {DA.material} {DA.solvent}, {float(DA.incident_angle[2:])}° Incidence',
                aspect='equal')
    ax.figure.set(tight_layout=True, dpi=130)
    
    # ax.figure.savefig(outPath.joinpath('recip_plots/stitched_v2', f'{DA.material}_{DA.solvent}_{DA.incident_angle}.png'), dpi=120)
    plt.show()
    plt.close('all')

#### Reciprocal Space Images

In [ ]:
filename = 'var_recip_stitched.zarr'
DS = xr.open_zarr(zarrsPath.joinpath(filename))
DS = DS.where(DS>1e-6)
DS

In [ ]:
selected_DAs = [da for da in DS.data_vars.values() if 
                da.attrs['incident_angle']]
len(selected_DAs)

In [ ]:
# Plot & optionally save each selected polymer: fix data set
for DA in tqdm(selected_DAs):
    # Slice data for selected q ranges (will need to rename q_xy if dimensions are differently named)
    sliced_DA = DA.sel(q_xy=slice(-1.1, 2.1), q_z=slice(-0.01, 2.2))
    # sliced_DA = DA.sel(q_xy=slice(-0.5, -0.25), q_z=slice(1.5, 1.75))

    # real_min = float(DA.sel(q_xy=slice(-0.5, -0.1), q_z=slice(0.1, 0.4)).compute().quantile(1e-3))
    real_min = float(DA.compute().quantile(0.01))
    cmin = 1 if real_min < 1 else real_min
    
    # cmax = float(DA.sel(q_xy=slice(-0.5, -0.1), q_z=slice(0.1, 2)).compute().quantile(1))   
    cmax = float(DA.compute().quantile(0.999))   
    
    # Same plotting procedure as above
    # ax = sliced_DA.plot.imshow(cmap=cmap, norm=LogNorm(cmin, cmax), interpolation='antialiased', figsize=(5.5,3.3))
    ax = sliced_DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), interpolation='antialiased', figsize=(5.5,3.3))

    ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)
    ax.axes.set(title=f'Cartesian Plot: {DA.material} {DA.solvent}, {float(DA.incident_angle[2:])}° Incidence',
                aspect='equal', xlabel='q$_{xy}$ [Å$^{-1}$]', ylabel='q$_z$ [Å$^{-1}$]')
    ax.figure.set(tight_layout=True, dpi=130)
    
    ax.figure.savefig(outPath.joinpath('recip_plots/stitched_v2', f'{DA.material}_{DA.solvent}_{DA.incident_angle}.png'), dpi=120)
    # plt.show()
    plt.close('all')

In [ ]:
# Plot & optionally save each selected polymer: var data set
for DA in tqdm(selected_DAs):
    # Slice data for selected q ranges (will need to rename q_xy if dimensions are differently named)
    sliced_DA = DA.sel(q_xy=slice(-1.1, 2.1), q_z=slice(-0.01, 2.2))
    # sliced_DA = DA.sel(q_xy=slice(-0.5, -0.25), q_z=slice(1.5, 1.75))

    # real_min = float(DA.sel(q_xy=slice(-0.5, -0.1), q_z=slice(0.1, 0.4)).compute().quantile(1e-3))
    real_min = float(DA.compute().quantile(0.01))
    cmin = 1 if real_min < 1 else real_min
    
    # cmax = float(DA.sel(q_xy=slice(-0.5, -0.25), q_z=slice(0.1, 1.75)).compute().quantile(1-1e-10))   
    cmax = float(DA.compute().quantile(0.999))   
    
    # Same plotting procedure as above
    ax = sliced_DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), interpolation='antialiased', figsize=(5.5,3.3))
    ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)
    ax.axes.set(title=f'Cartesian Plot: {DA.material} {DA.solvent} {DA.rpm}, {float(DA.incident_angle[2:])}° Incidence',
                aspect='equal', xlabel='q$_{xy}$ [Å$^{-1}$]', ylabel='q$_z$ [Å$^{-1}$]')
    ax.figure.set(tight_layout=True, dpi=130)
    
    ax.figure.savefig(outPath.joinpath('recip_plots/stitched_v2', f'{DA.material}_{DA.solvent}_{DA.rpm}_{DA.incident_angle}.png'), dpi=120)
    # plt.show()
    plt.close('all')

In [ ]:
# # A way to save data as csv files
# for DA in tqdm(DS.data_vars.values()):
#     # qxy columns, qz rows
#     DA.to_pandas().to_csv(outPath.joinpath('PM6-Y6_waxs', f'cartesian-2D_{DA.polymer}-{DA.weight_percent}_{DA.incident_angle}_{DA.scan_id}.csv'))

### 1D Plots

In [ ]:
filename = 'caked_PM6-Y6_waxs_stitched.zarr'
DS = xr.open_zarr(samplesPath.joinpath(filename))
DS = DS.where(DS>1e-5)
DS

In [ ]:
selected_DAs = [da for da in DS.data_vars.values() if 
                da.attrs['incident_angle'] == 'th0.120']
len(selected_DAs)

In [ ]:
fpath = outPath.joinpath('PM6-Y6set_waxs', f'linecut_IP_{DA.polymer}-{DA.weight_percent}_{chi_min}to{chi_max}chi_{DA.incident_angle}.csv')
OOP_linecut_DA.to_dataframe('OOP Intensity').to_csv(fpath)

In [ ]:
# Plot linecuts for selected chi ranges, here I've put both in plane and out of plane selections into the loop

for DA in tqdm(DS.data_vars.values()):
    # OOP
    chi_min = -18
    chi_max = -8
    OOP_linecut_DA = DA.sel(chi=slice(chi_min, chi_max), qr=slice(0.14,2.01)).sum('chi')
    OOP_linecut_DA.plot.line(figsize=(6,4))

    # A plot.line xarray plot does not return an AxesImage object like imshow does, so I use plt.gca() and plt.gcf() to access the axes & figure parameters
    ax = plt.gca()
    fig = plt.gcf()
    
    ax.set(title=f'OOP Linecut, {chi_min}° to {chi_max}° $\chi$: {DA.polymer}-{DA.weight_percent}, {float(DA.incident_angle[2:])}° Incidence',
           yscale='log', ylabel='Intensity [arb. units]', xlabel='q$_r$ [Å$^{-1}$]')
    ax.grid(visible=True, which='major', axis='x')
    fig.set(tight_layout=True, dpi=130)
    
    plt.show()
    fpath = outPath.joinpath('PM6-Y6set_waxs', f'linecut_OOP_{DA.polymer}-{DA.weight_percent}_{chi_min}to{chi_max}chi_{DA.incident_angle}.csv')
    print('Saving csv data...')
    OOP_linecut_DA.to_dataframe('OOP_Intensity').to_csv(fpath)
    print('Saved!')
    # fig.savefig(outPath.joinpath('bladecoated-set_waxs', f'linecut_OOP_{DA.polymer}-{DA.weight_percent}_{chi_min}to{chi_max}chi_{DA.incident_angle}.png'), dpi=150)
    plt.close('all')
    
    # IP
    chi_min = 72
    chi_max = 82
    IP_linecut_DA = DA.sel(chi=slice(chi_min, chi_max), qr=slice(0.23,2.01)).sum('chi')
    IP_linecut_DA.plot.line(figsize=(6,4))  
    
    ax = plt.gca()
    fig = plt.gcf()
    
    ax.set(title=f'IP Linecut, {chi_min}° to {chi_max}° $\chi$: {DA.polymer}-{DA.weight_percent}, {float(DA.incident_angle[2:])}° Incidence',
           yscale='log', ylabel='Intensity [arb. units]', xlabel='q$_r$ [Å$^{-1}$]')
    ax.grid(visible=True, which='major', axis='x')
    fig.set(tight_layout=True, dpi=130)
    
    plt.show()
    fpath = outPath.joinpath('PM6-Y6set_waxs', f'linecut_IP_{DA.polymer}-{DA.weight_percent}_{chi_min}to{chi_max}chi_{DA.incident_angle}.csv')
    print('Saving csv data...')
    IP_linecut_DA.to_dataframe('IP_Intensity').to_csv(fpath)  
    print('Saved!')
    # fig.savefig(outPath.joinpath('PM6-Y6set_waxs', f'linecut_IP_{DA.polymer}-{DA.weight_percent}_{chi_min}to{chi_max}chi_{DA.incident_angle}.png'), dpi=150)
    plt.close('all')
    